# poi_inf

In [0]:
def poi_format(cons):
    bid = cons[0]
    name = cons[2]
    std_tag = cons[11]
    aoi_id = cons[13]
    aoi_name = cons[14]
    point_x = cons[19]
    point_y = cons[20]
    parent_id = cons[31]
    return {'bid':bid, 'name':name, 'std_tag':std_tag, 
            'loc':(point_x,point_y), 'parent_id':parent_id,
           'aoi_id':aoi_id,' aoi_name':aoi_name}
import time
poi_file = '/home/bil/huangtao/project/data/poi_basic/poi_75/poi_75'
poi_dict = {}
flag = 1
st = time.time()
with open(poi_file) as f:
    for line in f:
        cons = eval(line)
        bid = cons[0]
        one_dict = poi_format(cons)
        poi_dict[bid] = one_dict
        if flag % 10000 == 0:
            t = time.time() - st
            print '%d / %d time:%.2f s\r' %(flag,851235,t*(851235-flag)/flag),
        flag += 1

# brand-relaton

In [0]:
file_path = '/home/bil/huangtao/project/data/competition/brand_cmp_0.05_2_final.csv'
brand_pair = []
with open(file_path) as f:
    flag = 0
    for line in f:
        flag += 1
        if flag == 1:continue
        line = line.strip('\n').split(',')
        bid1 = line[1]
        bid2 = line[2]
        if bid1 == '' or bid2 == '':continue
        brand_pair += [[bid1, bid2]]
print len(brand_pair)

10156


In [0]:
brand_all = set()
for i in brand_pair:
    brand_all.add(i[0])
    brand_all.add(i[1])
print 'brand_all', len(brand_all)

brand_all 2393


# name matching

In [0]:
brand_poi = {}
flag = 0
for name in brand_all:
    flag+=1
    for i in poi_dict:
        name1 = poi_dict[i]['name']
        if name in name1 or name1 in name:
            bid = poi_dict[i]['bid']
            if name not in brand_poi:
                brand_poi[name] = [bid]
            else:
                brand_poi[name] += [bid]
    print '%d / %d\r' %(flag,2393),

In [0]:
for i in brand_poi['真功夫']:print i,poi_dict[i]['name'],poi_dict[i]['std_tag']
print len(brand_poi)

4323626138220451795 真功夫包子店(红光路) 美食;小吃快餐店
5201825851739398711 真功夫(成都双流国际机场T2航站楼店) 美食;小吃快餐店
895801163033462987 真功夫(成都龙湖北城天街店) 美食;小吃快餐店
2428516810711540962 成都真功夫冷冻批发部 购物;市场
12194047747762866253 真功夫(成都华西医院店) 美食;小吃快餐店
13807817782787410156 真功夫按摩浴足 休闲娱乐;洗浴按摩
10453799233739225300 真功夫(成都世豪广场店) 美食;小吃快餐店
16440067407962156241 真功夫理疗馆 休闲娱乐
8329784639214453874 外婆家真功夫烤鱼店 美食;中餐厅
2020


In [0]:
flag = 0
brand_poi_filter = {}
for brand in brand_poi:
    flag+=1
    tag_dict = {}
    for i in brand_poi[brand]:
        std_tag = poi_dict[i]['std_tag']
        if std_tag in tag_dict:
            tag_dict[std_tag] += 1
        else:
            tag_dict[std_tag] = 1
    all_tag = sorted(tag_dict.iteritems(),key = lambda x:x[1], reverse=True)
    for i in range(len(all_tag)):
        if '出入口' not in all_tag[i][0] and '停车' not in all_tag[i][0]:
            main_tag = all_tag[i][0]
            break
        else:
            main_tag = '0000000000'
    poi_list = []
    for i in brand_poi[brand]:
        std_tag = poi_dict[i]['std_tag']
        if std_tag == main_tag:
            poi_list += [i]
    brand_poi_filter[brand] = poi_list
    if flag % 100 == 0:print '%d / %d\r' %(flag,len(brand_poi)),

In [0]:
competition = set()
flag = 0
for line in brand_pair:
    flag+=1
    brand = line[0]
    compe = line[1]
    match = []
    if compe in brand_poi_filter:
        match = brand_poi_filter[compe]
    if len(match) == 0:
        continue
    try:
        bids = brand_poi_filter[brand]
    except:
        continue
    for i in match:
        for j in bids:
            new1 = i + '#' + j
            new2 = j + '#' + i
            competition.add(new1)
            competition.add(new2)
    print '%d / %d\r' %(flag,10156),

In [0]:
print len(competition)

12713413


In [0]:
bid_all = set()
for pair in competition:
    bid1 = pair.split('#')[0]
    bid2 = pair.split('#')[1]
    bid_all.add(bid1)
    bid_all.add(bid2)
print len(bid_all)

23689


In [0]:
import pickle
with open('/home/bil/huangtao/project/data/competition/data_75/bid_set.pickle', 'wb') as f:
        pickle.dump(bid_all, f)

# 筛选

In [0]:
bayes_file = '/home/bil/huangtao/project/data/competition/data_75/201809/part-00000'
bayes_dict = {}
with open(bayes_file) as f:
    for line in f:
        line = eval(line)
        bid = line[0]
        cuids = set(line[1])
        if bid not in bayes_dict:
            bayes_dict[bid] = cuids
        else:
            for cuid in cuids:
                bayes_dict[bid].add(cuid)

In [0]:
flag = 0
share_visit = {}
for pair in competition:
    flag+=1
    bid1 = pair.split('#')[0]
    bid2 = pair.split('#')[1]
    if bid1 == bid2:
        continue
    try:
        cuid1 = bayes_dict[bid1]
        cuid2 = bayes_dict[bid2]
    except:
        continue
    num = len(cuid1 & cuid2)
    if num !=0:
        if bid2+'#'+bid1 not in share_visit:
            share_visit[pair] = num
    if flag %1000 == 0:
        print '%d / %d\r' %(flag,len(competition)),

In [0]:
print len(share_visit)
#去银行
share_visit_filter = {}
for i in share_visit:
    if '银行' not in poi_dict[i.split('#')[0]]['name']:
        share_visit_filter[i] = share_visit[i]
print len(share_visit_filter)

46295
46074


In [0]:
#筛10000范围和tag
from shapely.geometry import Point
share_visit_filter_10000 = {}
flag = 0
for i in share_visit_filter:
    flag+=1
    if i.split('#')[0] == i.split('#')[1]:continue
    bid1 = i.split('#')[0]
    bid2 = i.split('#')[1]
    p1 = poi_dict[i.split('#')[0]]['loc']
    p2 = poi_dict[i.split('#')[1]]['loc']
    pp1 = [float(x) for x in p1]
    pp2 = [float(x) for x in p2]
    point1 = Point(pp1)
    point2 = Point(pp2)
    l = point1.distance(point2)
    std_tag1 = poi_dict[bid1]['std_tag']
    std_tag2 = poi_dict[bid2]['std_tag']
    if l < 10000 and std_tag1 == std_tag2:
        share_visit_filter_10000[i] = share_visit_filter[i]
    if flag % 1000 == 0:
        print '%d / %d\r' %(flag,len(share_visit_filter)),

In [0]:
print len(share_visit_filter_10000)

17234


In [0]:
#筛tag和占比
share_visit_res = {}
for pair in share_visit_filter_10000:
    bid1 = pair.split('#')[0]
    bid2 = pair.split('#')[1]
    visit_num = share_visit_filter_10000[pair]
    all_visit = len(bayes_dict[bid1] | bayes_dict[bid2])
    rate = float(visit_num) / all_visit
    if visit_num > 1:
        share_visit_res[pair] = share_visit_filter_10000[pair]
print len(share_visit_res)

8970


In [0]:
bid_res = set()#8月3055
for pair in share_visit_res:
    bid1 = pair.split('#')[0]
    bid2 = pair.split('#')[1]
    bid_res.add(bid1)
    bid_res.add(bid2)
print len(bid_res)

2481


In [0]:
from shapely.geometry import Point
ll = {}
for i in share_visit_res:
    if i.split('#')[0] == i.split('#')[1]:continue
    p1 = poi_dict[i.split('#')[0]]['loc']
    p2 = poi_dict[i.split('#')[1]]['loc']
    pp1 = [float(x) for x in p1]
    pp2 = [float(x) for x in p2]
    point1 = Point(pp1)
    point2 = Point(pp2)
    l = point1.distance(point2)
    #if '银行' not in poi_dict[i.split('#')[0]]['name']:
    rang = str(int(l/100))
    if l < 10000:
    #if True:
        if rang not in ll:
            ll[rang] = 1
        else:
            ll[rang] += 1
from pyecharts import Line
line = Line('距离分布')
ll_sort = sorted(ll.iteritems(),key = lambda x:int(x[0]))
attr = [x[0] for x in ll_sort]
v = [x[1] for x in ll_sort]
line.add('',attr,v)
line

In [0]:
tag_dist = {}
for i in share_visit_res:
    if i.split('#')[0] == i.split('#')[1]:continue
    p1 = poi_dict[i.split('#')[0]]['std_tag']
    p2 = poi_dict[i.split('#')[1]]['std_tag']
    if p1 not in tag_dist:
        tag_dist[p1] = 1
    else:
        tag_dist[p1]+=1
    if p2 not in tag_dist:
        tag_dist[p2] = 1
    else:
        tag_dist[p2]+=1
from pyecharts import Bar
line = Bar('tag分布')
ll_sort = sorted(tag_dist.iteritems(),key = lambda x:int(x[1]),reverse=True )[:10]
attr = [x[0] for x in ll_sort]
v = [x[1] for x in ll_sort]
line.add('',attr,v,xaxis_rotate=30)
line

# 保存

In [0]:
import pickle
with open('/home/bil/huangtao/project/data/competition/data_75/share_visit.pickle','wb') as f:
    pickle.dump(share_visit_res,f)
import pickle
with open('/home/bil/huangtao/project/data/competition/data_75/bids.pickle','wb') as f:
    pickle.dump(bid_res,f)

In [0]:
share_visit = share_visit_res
bid_all = bid_res

# 读取

In [0]:
import pickle
from shapely.geometry import Point,Polygon
with open('/home/bil/huangtao/project/data/competition/data_75/share_visit.pickle','rb') as f:
    share_visit = pickle.load(f)
import pickle
with open('/home/bil/huangtao/project/data/competition/data_75/bids.pickle','rb') as f:
    bid_all = pickle.load(f)

# 反例

In [0]:
count_tag_dict = {}
for pair in share_visit:
    bid1 = pair.split('#')[0]
    bid2 = pair.split('#')[1]
    std_tag1 = poi_dict[bid1]['std_tag']
    if std_tag1 not in count_tag_dict:
        count_tag_dict[std_tag1] = 1
    else:
        count_tag_dict[std_tag1] += 1
sort_data = sorted(count_tag_dict.iteritems(),key = lambda x:int(x[1]),reverse=True)
for i in sort_data:print i[0],i[1]

酒店;快捷酒店 3946
交通设施;加油加气站 1835
美食;小吃快餐店 1501
生活服务;物流公司 454
生活服务;通讯营业厅 447
购物;商铺 256
购物;家电数码 139
购物;超市 104
美食;咖啡厅 95
美食;蛋糕甜品店 57
购物;家居建材 49
购物;便利店 31
生活服务;房产中介机构 12
美食;中餐厅 10
金融;投资理财 10
汽车服务;汽车销售 8
丽人;美发 5
生活服务;彩票销售点 5
汽车服务;汽车租赁 2
运动健身;健身中心 1
休闲娱乐;电影院 1
丽人;美容 1
汽车服务;汽车配件 1


In [0]:
#构建所有反例
negtive_all = []
for bid1 in bid_all:
    for bid2 in bid_all:
        if bid1 == bid2:continue
        tag1 = poi_dict[bid1]['std_tag']
        tag2 = poi_dict[bid2]['std_tag']
        if tag1 == tag2:
            negtive_all += [bid1 + '#' + bid2]
print len(negtive_all)

725604


In [0]:
import random
from shapely.geometry import Point,Polygon
negtive_all = random.sample(negtive_all,len(negtive_all))
flag = 0
negtive = []
negtive_tag_dict = {x:0 for x in count_tag_dict}
flag = 0
for pair in negtive_all:
    flag+=1
    bid1 = pair.split('#')[0]
    bid2 = pair.split('#')[1]
    std_tag1 = poi_dict[bid1]['std_tag']
    #筛分布
    if negtive_tag_dict[std_tag1] >= count_tag_dict[std_tag1]:continue
    #筛正例
    if bid1 + '#' + bid2 in share_visit or bid2 + '#' + bid1 in share_visit:
        continue
    #筛距离
    p1 = poi_dict[bid1]['loc']
    p2 = poi_dict[bid2]['loc']
    pp1 = [float(x) for x in p1]
    pp2 = [float(x) for x in p2]
    point1 = Point(pp1)
    point2 = Point(pp2)
    l = point1.distance(point2)
    if l > 10000:continue
    #筛co-visit
    try:
        cuid1 = bayes_dict[bid1]
        cuid2 = bayes_dict[bid2]
    except:
        continue
    num = len(cuid1 & cuid2)
    if num > 2:
        continue
    #筛brand
    name1 = poi_dict[bid1]['name'].split('(')[0]
    name2 = poi_dict[bid2]['name'].split('(')[0]
    brand1 = ''
    brand2 = ''
    for brand in brand_all:
        if name1 in brand or brand in name1:
            brand1 = brand
        if name2 in brand or brand in name2:
            brand2 = brand
        if brand1 != '' and brand2 != '':
            break
    if brand1 == brand2:
        continue
    b = brand1 + '#' + brand2
    if b not in pair_set:
        if bid1+'#'+bid2 not in negtive:
            negtive_tag_dict[std_tag1] += 1
            negtive += [bid2+'#'+bid1]
    if flag % 100 == 0:
        print '\r %d / %d' %(flag, len(negtive_all)),

 377900 / 725604                                  


In [0]:
sort_data = sorted(negtive_tag_dict.iteritems(),key = lambda x:int(x[1]),reverse=True)
for i in sort_data:print i[0],i[1]

酒店;快捷酒店 3946
美食;小吃快餐店 1501
生活服务;物流公司 454
生活服务;通讯营业厅 447
交通设施;加油加气站 300
购物;商铺 256
购物;家电数码 139
购物;超市 104
美食;咖啡厅 95
美食;蛋糕甜品店 57
购物;家居建材 49
购物;便利店 31
生活服务;房产中介机构 11
美食;中餐厅 10
金融;投资理财 10
丽人;美发 5
生活服务;彩票销售点 5
汽车服务;汽车销售 3
运动健身;健身中心 0
休闲娱乐;电影院 0
汽车服务;汽车租赁 0
丽人;美容 0
汽车服务;汽车配件 0


In [0]:
print len(negtive)

7423


In [0]:
import pickle
with open('/home/bil/huangtao/project/data/competition/data_75/negtive.pickle','wb') as f:
    pickle.dump(negtive,f)

In [0]:
import pickle
with open('/home/bil/huangtao/project/data/competition/data_75/negtive.pickle','rb') as f:
    negtive = pickle.load(f)

# region-feature

In [0]:
path = '/home/bil/huangtao/project/data/region/town.csv.utf8'
import numpy as np

In [0]:
town_dict = {}
no = 0
with open(path) as f:
    for line in f:
        line = line.strip('\n').split('\t')
        pro = line[3]
        city = line[2]
        if '四川' in pro:
            polygon = line[0]
            polygon = eval(polygon)
            polygon = polygon[8:]
            polygon = polygon.replace(' ',',')
            polygon = eval(polygon)
            if type(polygon[0]) == float:
                polygon = [polygon]
            res = []
            for poly in polygon:
                re = [[x,y] for x,y in zip(poly[::2],poly[1:][::2])]
                res+=[re]
            town_id = line[1]
            name = line[2]
            if name == '':
                name = str(no)
                no+=1
            c = line[4]
            s = line[5]
            town_dict[town_id] = [name,c,s,res]
print len(town_dict)

4445


# poi-town-dict

In [0]:
#-----------poi-town-dict---------
from shapely.geometry import Point,Polygon
bid_town_dict = {}
flag = 0
for bid in bid_all:
    flag+=1
    loc = poi_dict[bid]['loc']
    loc = [float(x) for x in loc]
    point = Point(loc)
    for town_id in town_dict:
        polygons = town_dict[town_id][3]
        for polygon in polygons:
            p = Polygon(polygon)
            if p.contains(point):
                if bid not in bid_town_dict:
                    bid_town_dict[bid] = [town_id]
                else:
                    bid_town_dict[bid] += [town_id]
            break
    print '%d / %d\r' %(flag,len(bid_all)),    

In [0]:
len(bid_town_dict)

2481

In [0]:
import pickle
with open('/home/bil/huangtao/project/data/competition/data_75/bid_town_dict.pickle','wb') as f:
    pickle.dump(bid_town_dict,f)

# 精简town_dict

In [0]:
town_set = set()
for bid in bid_town_dict:
    towns = bid_town_dict[bid]
    for town in towns:
        town_set.add(town)
print len(town_set)

177


In [0]:
town_dict_filter = dict()
for town in town_set:
    town_dict_filter[town] = town_dict[town]
print len(town_dict_filter)

177


In [0]:
town_dict = town_dict_filter

In [0]:
import pickle
with open('/home/bil/huangtao/project/data/competition/data_75/town_dict.pickle','wb') as f:
    pickle.dump(town_dict,f)#{town_id:[bids]}

In [0]:
import pickle
with open('/home/bil/huangtao/project/data/competition/data_75/town_dict.pickle','rb') as f:
    town_dict = pickle.load(f)#{town_id:[bids]}

# 统计所有下属poi

In [0]:
from shapely.geometry import Point,Polygon
town_all_bid = {}
flag = 0
st = time.time()
for town_id in town_dict:
    flag+=1
    polygons = town_dict[town_id][3]
    for polygon in polygons:
        p = Polygon(polygon)
        for bid in poi_dict:
            loc = poi_dict[bid]['loc']
            loc = [float(x) for x in loc] 
            point = Point(loc)
            if p.contains(point):
                if town_id not in town_all_bid:
                    town_all_bid[town_id] = [bid]
                else:
                    town_all_bid[town_id] += [bid]
    print '%d / %d all:%.2fs cost:%.2fs\r' %(flag,len(town_dict),(time.time()-st)*len(town_dict)/flag,time.time()-st),   

In [0]:
import pickle
with open('/home/bil/huangtao/project/data/competition/data_75/town_all_bid.pickle','wb') as f:
    pickle.dump(town_all_bid,f)#{town_id:[bids]}

In [0]:
import pickle
with open('/home/bil/huangtao/project/data/competition/data_75/town_all_bid.pickle','rb') as f:
    town_all_bid = pickle.load(f)#{town_id:[bids]}

# feature from poi_basic

In [0]:
flag = 0
region_feature = {}
for town_id in town_all_bid:
    flag += 1
    bids = town_all_bid[town_id]
    town_feature = {}
    for bid in bids:
        std_tag = poi_dict[bid]['std_tag']
        for tag in std_tag.split(';'):
            if tag == '':
                continue
            tag = tag + '_num'
            if tag not in town_feature:
                town_feature[tag] = 1
            else:
                town_feature[tag] += 1
    town_feature['poi_num'] = len(bids)
    town_feature['c'] = town_dict[town_id][1]
    town_feature['s'] = town_dict[town_id][2]
    region_feature[town_id] = town_feature
    print '%d / %d\r' %(flag,len(town_all_bid)), 

In [0]:
import pickle
with open('/home/bil/huangtao/project/data/competition/data_75/region_feature.pickle','wb') as f:
    pickle.dump(region_feature,f)#{town_id:[bids]}

In [0]:
import pickle
with open('/home/bil/huangtao/project/data/competition/data_75/region_feature.pickle','rb') as f:
    region_feature = pickle.load(f)#{town_id:[bids]}

In [0]:
len( region_feature)

177

# graph-prepare

In [0]:
import gzip
path = '/home/bil/huangtao/project/data/co_data/cd_co_map/201808/'
co_map = {}
flag = 0
for part in range(100):
    flag += 1
    p = path + 'part-%05d.gz' % part
    with gzip.open(p, 'rb') as f:
        for line in f:
            line = line.split('\t')
            bid1 = line[0]
            bid2 = line[1]
            val = line[2]
            key = bid1 + '#' + bid2
            if bid1 in poi_dict and bid2 in poi_dict:
                co_map[key] = val
    print '%d / %d\r' %(flag,100),

In [0]:
print len(co_map)

NameError: name 'co_map' is not defined

In [0]:
co_map_f = {}
flag = 0
for key in co_map:
    flag += 1
    val = co_map[key].strip('\n')
    val = int(val)
    if val > 10:
        co_map_f[key] = val
    if flag %100000 == 0:print '\r %d / %d' %(flag, len(co_map)),

 74400000 / 74427278                                                                                                                                                                                                                                                                                                                                                             


In [0]:
print len(co_map_f)
co_map = {}

18350772


In [0]:
co_map_filter = {}
flag = 0
from shapely.geometry import Point
ll = len(co_map_f)
for pair in co_map_f:
    flag += 1
    bid1 = pair.split('#')[0]
    bid2 = pair.split('#')[1]
    pair1 = bid2 + '#' + bid1
    try:
        p1 = poi_dict[bid1]['loc']
        p2 = poi_dict[bid2]['loc']
    except:
        continue
    pp1 = [float(x) for x in p1]
    pp2 = [float(x) for x in p2]
    point1 = Point(pp1)
    point2 = Point(pp2)
    l = point1.distance(point2)
    if l > 10000:continue
    if pair not in co_map_filter and pair1 not in co_map_filter:
        co_map_filter[pair] = co_map_f[pair]
    if flag % 100 == 0:
        print '%d / %d  filter:%d\r' %(flag,ll,len(co_map_filter)), 

In [0]:
import pickle
with open('/home/bil/huangtao/project/data/competition/data_75/co_map_filter.pickle','wb') as f:
    pickle.dump(co_map_filter,f)#{town_id:[bids]}

In [0]:
import pickle
with open('/home/bil/huangtao/project/data/competition/data_75/co_map_filter.pickle','rb') as f:
    co_map_filter = pickle.load(f)#{town_id:[bids]}

# sfe


In [0]:
import os
import time
import pandas as pd
import numpy as np
import itertools


class Edge(object):
    def __init__(self, start, end, label):
        self.label = str(label)
        self.start = start
        self.end = end
        self.str = self.__str__()

    def __str__(self):
        return self.label 


class Node(object):
    def __init__(self, name):
        self.name = name
        self.neighbors = set()
        self.neighbor2edges = {} # a dict of the form {node: [edge1, edge2, ...]}
        self.neighbor2edgesstr = {} # a dict of the form {node: [edge1.str, edge2.str, ...]}
        
    def __str__(self):
        return "Node({})".format(self.name)

    def add_edge(self, edge):
        self.neighbors.add(edge.end)
        self.neighbor2edges[edge.end] = self.neighbor2edges.get(edge.end, []) + [edge]
        self.neighbor2edgesstr[edge.end] = self.neighbor2edgesstr.get(edge.end, []) + [edge.str]
        
class Graph(object):
    def __init__(self):
        self.nodes = {} # nodes are indexed by name

    def get_node(self, name, create=False):
        if create and not name in self.nodes:
            self.nodes[name] = Node(name)
        return self.nodes[name]

    def partial_build_from_df(self, df):
        """Arguments:
        - `df` (pandas.DataFrame): DataFrame with three columns: head, tail and relation.
        Each row represents a triple in a knowledge graph.
        """
        flag = 0
        l = len(df)
        for idx,row in df.iterrows():
            flag += 1
            head = self.get_node(row['head'], create=True)
            tail = self.get_node(row['tail'], create=True)
            relation = row['relation']
            head.add_edge(Edge(head, tail, relation))
            tail.add_edge(Edge(tail, head, relation))
            if flag % 1000 == 0:
                print '%d / %d\r' %(flag, l),
    def partial_build_from_list(self, data_list):
        """Arguments:
        - `df` (pandas.DataFrame): DataFrame with three columns: head, tail and relation.
        Each row represents a triple in a knowledge graph.
        """
        flag = 0
        l = len(data_list)
        for pair in data_list:
            flag += 1
            bid1 = line[0]
            val = line[1]
            bid2 = line[2]
            head = self.get_node(bid1, create=True)
            tail = self.get_node(bid2, create=True)
            relation = val
            head.add_edge(Edge(head, tail, relation))
            tail.add_edge(Edge(tail, head, relation))
            if flag % 1000 == 0:
                print '%d / %d\r' %(flag, l),

In [0]:
import random
class SFE(object):
    def __init__(self, graph):
        self.graph = graph
    
    def get_paths(self, node_seqs):
        paths = []
        for node_seq in node_seqs:
            possible_paths = []
            for i in range(1, len(node_seq)):
                possible_paths += [node_seq[i-1].neighbor2edgesstr[node_seq[i]][0]]
            paths += [possible_paths]
        return paths

    def get_4len_path(self, head_name, tail_name, n):
        head = self.graph.nodes[head_name]
        tail = self.graph.nodes[tail_name]
        tail_nears = tail.neighbors
        head_nears = head.neighbors
        head_nears = random.sample(list(head_nears),len(head_nears))
        seq = []
        for head_near in head_nears:
            head_near_nears = head_near.neighbors
            for head_near_near in head_near_nears:
                if head_near_near in tail_nears:
                    seq += [[head, head_near, head_near_near, tail]]
                    if len(seq) >= n:
                        break
            if len(seq) >= n:
                break
        if seq == []:
            return seq, seq
        else:
            nodes = []
            edges = []
            for one_seq in seq:
                one_nodes = []
                for node in one_seq:
                    one_nodes += [node.name]
                one_edges = []
                for i in range(1, len(one_seq)):
                    one_edges += [int(one_seq[i-1].neighbor2edgesstr[one_seq[i]][0].strip('\n'))]
                nodes += [one_nodes]
                edges += [one_edges]
            return nodes
    
    def get_n_path(self, head_name, tail_name, n):
        head = self.graph.nodes[head_name]
        tail = self.graph.nodes[tail_name]
        tail_nears = tail.neighbors
        head_nears = head.neighbors
        head_nears = random.sample(list(head_nears),len(head_nears))
        def get_path(head_nears, tail_nears):
            head_nears = random.sample(list(head_nears),len(head_nears))
            path1 = []
            for head_near in head_nears:
                if head_near in tail_nears:
                    path1 = [head, head_near, tail]
                    break
            if path1 != []:
                path1 = [x.name for x in path1]
            return path1
        
        path = []
        for i in range(n):
            path += [get_path(head_nears, tail_nears)]
        return path
        
    def sample_3len_path(self, head_name, tail_name, n):
        head = self.graph.nodes[head_name]
        tail = self.graph.nodes[tail_name]
        tail_nears = tail.neighbors
        head_nears = head.neighbors
        head_nears = random.sample(list(head_nears),len(head_nears))
        temp = []
        for head_near in head_nears:
            if head_near in tail_nears:
                temp += [head_near]
                if len(temp) == n:
                    break
        res = []
        for node in temp:
            t = [head.name, tail.name, node.name]
            res += [t]
        if len(res) != n:
            last = n - len(res)
            res += [[]] * last
        return res
    
    def get_ordinary_3len_path(self, head_name, tail_name, n, ordinary_dict, m):#n个path，搜索m次
        head = self.graph.nodes[head_name]
        tail = self.graph.nodes[tail_name]
        tail_nears = tail.neighbors
        head_nears = head.neighbors
        tag = get_tag1(head_name)
        
        head_nears = list(head_nears)
        temp = []
        for i in range(m):
            node = random.sample(head_nears,1)[0]
            if node not in tail_nears:continue
            key  = tag + '#' + get_tag1(node.name)
            if key in ordinary_dict:
                ordinary_dict[key] += 1
                if len(temp) != n:
                    temp += [node]
        res = []
        for node in temp:
            t = [head.name, tail.name, node.name]
            res += [t]       
        if len(res) != n:
            last = n - len(res)
            res += [[]] * last
        return res, ordinary_dict
    
    def get_4node_motif(self, head_name, tail_name, nodes_3len, nodes_4len):
        head = self.graph.nodes[head_name]
        tail = self.graph.nodes[tail_name]
        tail_nears = tail.neighbors
        head_nears = head.neighbors
        head_nears = random.sample(list(head_nears),len(head_nears))
        #motif1:2len_path_extend
        motif1 = []
        if head in tail_nears:
            temp = []
            for head_near in head_nears:
                if head_near in tail_nears:
                    temp += [head_near]
                    if len(temp) == 2:break
            if len(temp) == 2:
                motif1 = [head, tail, temp[0], temp[1]]
            elif len(temp) == 1:
                motif1 = [head, tail, temp[0], random.sample(list(tail_nears),1)[0]]
            elif len(temp) == 0:
                motif1 = [head, tail, head_nears[0], random.sample(list(tail_nears),1)[0]]
            if motif1 != []:
                temp = motif1
                motif1 = [x.name for x in temp]
        #motif2:3len_path_extend
        if nodes_3len == []:
            motif2 = []
        else:
            nodes = nodes_3len[0]
            middle = self.graph.nodes[nodes[1]]
            middle_nears = middle.neighbors
            for head_near in head_nears:
                if head_near in tail_nears and head_near in middle_nears:
                    motif2 = [head, middle, tail, head_near]
                elif head_near in tail_nears or head_near in middle_nears:
                    motif2 = [head, middle, tail, head_near]
            if motif2 == []:
                tail_nears = random.sample(list(tail_nears),len(tail_nears))
                for tail_near in tail_nears:
                    if tail_near in head_nears and tail_near in middle_nears:
                        motif2 = [head, middle, tail, head_near]
                    elif tail_near in head_nears or tail_near in middle_nears:
                        motif2 = [head, middle, tail, head_near]
            if motif2 == []:
                motif2 = [head, middle, tail, head_nears[0]]
            if motif2 != []:
                temp = motif2
                motif2 = [x.name for x in temp]
        #motif3:4len_path_extend
        if nodes_4len == []:
            motif3 = []
        else:
            motif3 = nodes_4len[0]
        return motif1, motif2, motif3
    
    def get_2n_motif(self, head_name, tail_name, n):
        head = self.graph.nodes[head_name]
        tail = self.graph.nodes[tail_name]
        tail_nears = tail.neighbors
        head_nears = head.neighbors
        head_nears = random.sample(list(head_nears),len(head_nears))
        #motif1:2len_path_extend
        def get_motif2(head_nears, tail_nears):
            head_nears = random.sample(list(head_nears),len(head_nears))
            motif1 = []
            if head in tail_nears:
                temp = []
                for head_near in head_nears:
                    if head_near in tail_nears:
                        temp += [head_near]
                        if len(temp) == 2:break
                if len(temp) == 2:
                    motif1 = [head, tail, temp[0], temp[1]]
                elif len(temp) == 1:
                    motif1 = [head, tail, temp[0], random.sample(list(tail_nears),1)[0]]
                elif len(temp) == 0:
                    motif1 = [head, tail, head_nears[0], random.sample(list(tail_nears),1)[0]]
                if motif1 != []:
                    temp = motif1
                    motif1 = [x.name for x in temp]
            return motif1
        
        motif = []
        for i in range(n):
            motif += [get_motif2(head_nears, tail_nears)]
        #motif2:3len_path_extend
        def get_motif3(head_nears, tail_nears):
            head_nears = random.sample(list(head_nears),len(head_nears))
            seq = []
            for head_near in head_nears:
                if head_near in tail_nears:
                    seq = [head, head_near, tail]
                    seq = [x.name for x in seq]
                    break
            motif2 = []
            if seq == []:
                motif2 = []
            else:
                nodes = seq
                middle = self.graph.nodes[nodes[1]]
                middle_nears = middle.neighbors
                for head_near in head_nears:
                    if head_near in tail_nears and head_near in middle_nears:
                        motif2 = [head, middle, tail, head_near]
                    elif head_near in tail_nears or head_near in middle_nears:
                        motif2 = [head, middle, tail, head_near]
                if motif2 == []:
                    tail_nears = random.sample(list(tail_nears),len(tail_nears))
                    for tail_near in tail_nears:
                        if tail_near in head_nears and tail_near in middle_nears:
                            motif2 = [head, middle, tail, head_near]
                        elif tail_near in head_nears or tail_near in middle_nears:
                            motif2 = [head, middle, tail, head_near]
                if motif2 == []:
                    motif2 = [head, middle, tail, head_nears[0]]
                if motif2 != []:
                    temp = [motif2[0], motif2[2], motif2[1], motif2[3]]
                    motif2 = [x.name for x in temp]
            return motif2
        for i in range(n):
            motif += [get_motif3(head_nears, tail_nears)]
        return motif

    def sample_motif1(self, head_name, tail_name, n):#左三角
        head = self.graph.nodes[head_name]
        tail = self.graph.nodes[tail_name]
        tail_nears = tail.neighbors
        head_nears = head.neighbors
        head_nears = random.sample(list(head_nears),len(head_nears))
        tail_nears = random.sample(list(tail_nears),len(tail_nears))
        if head not in tail_nears:
            return [[]]*n
        else:
            temp = []
            for head_near in head_nears:
                if head_near not in tail_nears:
                    temp += [head_near]
                if len(temp) == n:
                    break
            res = []
            for node in temp:
                t = [head.name, tail.name, node.name]
                res += [t]
            if len(res) != n:
                last = n - len(res)
                res += [[]] * last
            return res
    
    def get_ordinary_motif1(self, head_name, tail_name, ordinary_dict, m, tag_key):#n个motif，搜索m次
        head = self.graph.nodes[head_name]
        tail = self.graph.nodes[tail_name]
        tail_nears = tail.neighbors
        head_nears = head.neighbors
        tag = get_tag1(head_name)
        if head not in tail_nears:
            res = [[]]*len(tag_key)
        else:
            head_nears = list(head_nears)
            temp = [[]] * len(tag_key)
            for i in range(m):
                node = random.sample(head_nears,1)[0]
                if node in tail_nears:continue
                key  = tag + '#' + get_tag1(node.name)
                if key in ordinary_dict:
                    ordinary_dict[key] += 1
                for j in range(len(tag_key)):
                    if key == tag_key[j]:
                        temp[j] = node
            res = []
            for node in temp:
                if node != []:
                    t = [head.name, tail.name, node.name]
                    res += [t]   
                else:
                    res += [[]]
        return res, ordinary_dict
    
    def sample_motif2(self, head_name, tail_name, n):#右三角
        head = self.graph.nodes[head_name]
        tail = self.graph.nodes[tail_name]
        tail_nears = tail.neighbors
        head_nears = head.neighbors
        head_nears = random.sample(list(head_nears),len(head_nears))
        tail_nears = random.sample(list(tail_nears),len(tail_nears))
        if head not in tail_nears:
            return [[]]*n
        else:
            temp = []
            for tail_near in tail_nears:
                if tail_near not in head_nears:
                    temp += [tail_near]
                if len(temp) == n:
                    break
            res = []
            for node in temp:
                t = [head.name, tail.name, node.name]
                res += [t]
            if len(res) != n:
                last = n - len(res)
                res += [[]] * last
            return res
    
    def get_ordinary_motif2(self, head_name, tail_name, ordinary_dict, m, tag_key):#n个motif，搜索m次
        head = self.graph.nodes[head_name]
        tail = self.graph.nodes[tail_name]
        tail_nears = tail.neighbors
        head_nears = head.neighbors
        tag = get_tag1(head_name)
        if head not in tail_nears:
            res = [[]]*len(tag_key)
        else:
            tail_nears = list(tail_nears)
            temp = [[]] * len(tag_key)
            for i in range(m):
                node = random.sample(tail_nears,1)[0]
                if node in head_nears:continue
                key  = tag + '#' + get_tag1(node.name)
                if key in ordinary_dict:
                    ordinary_dict[key] += 1
                for j in range(len(tag_key)):
                    if key == tag_key[j]:
                        temp[j] = node
            res = []
            for node in temp:
                if node != []:
                    t = [head.name, tail.name, node.name]
                    res += [t]  
                else:
                    res += [[]]
        return res, ordinary_dict
    
    def sample_motif3(self, head_name, tail_name, n):#上三角
        head = self.graph.nodes[head_name]
        tail = self.graph.nodes[tail_name]
        tail_nears = tail.neighbors
        head_nears = head.neighbors
        head_nears = random.sample(list(head_nears),len(head_nears))
        tail_nears = random.sample(list(tail_nears),len(tail_nears))
        
        if head in tail_nears:
            return [[]]*n
        else:
            temp = []
            for head_near in head_nears:
                if head_near in tail_nears:
                    temp += [head_near]
                    if len(temp) == n:
                        break
            res = []
            for node in temp:
                t = [head.name, tail.name, node.name]
                res += [t]
            if len(res) != n:
                last = n - len(res)
                res += [[]] * last
            return res  
        
    def get_ordinary_motif3(self, head_name, tail_name, ordinary_dict, m, tag_key):#n个motif，搜索m次
        head = self.graph.nodes[head_name]
        tail = self.graph.nodes[tail_name]
        tail_nears = tail.neighbors
        head_nears = head.neighbors
        tag = get_tag1(head_name)
        if head in tail_nears:
            res = [[]]*len(tag_key)
        else:
            head_nears = list(head_nears)
            temp = [[]] * len(tag_key)
            for i in range(m):
                node = random.sample(head_nears,1)[0]
                if node not in tail_nears:continue
                key  = tag + '#' + get_tag1(node.name)
                if key in ordinary_dict:
                    ordinary_dict[key] += 1
                for j in range(len(tag_key)):
                    if key == tag_key[j]:
                        temp[j] = node
            res = []
            for node in temp:
                if node != []:
                    t = [head.name, tail.name, node.name]
                    res += [t] 
                else:
                    res += [[]]
        return res, ordinary_dict
    
    def sample_motif4(self, head_name, tail_name, n):#全三角
        head = self.graph.nodes[head_name]
        tail = self.graph.nodes[tail_name]
        tail_nears = tail.neighbors
        head_nears = head.neighbors
        head_nears = random.sample(list(head_nears),len(head_nears))
        tail_nears = random.sample(list(tail_nears),len(tail_nears))
        if head not in tail_nears:
            return [[]]*n
        else:
            temp = []
            for head_near in head_nears:
                if head_near in tail_nears:
                    temp += [head_near]
                    if len(temp) == n:
                        break
            res = []
            for node in temp:
                t = [head.name, tail.name, node.name]
                res += [t]
            if len(res) != n:
                last = n - len(res)
                res += [[]] * last
            return res
    
    def get_ordinary_motif4(self, head_name, tail_name, ordinary_dict, m, tag_key):#n个motif，搜索m次
        head = self.graph.nodes[head_name]
        tail = self.graph.nodes[tail_name]
        tail_nears = tail.neighbors
        head_nears = head.neighbors
        tag = get_tag1(head_name)
        if head not in tail_nears:
            res = [[]]*len(tag_key)
        else:
            head_nears = list(head_nears)
            temp = [[]] * len(tag_key)
            for i in range(m):
                node = random.sample(head_nears,1)[0]
                if node not in tail_nears:continue
                key  = tag + '#' + get_tag1(node.name)
                if key in ordinary_dict:
                    ordinary_dict[key] += 1
                for j in range(len(tag_key)):
                    if key == tag_key[j]:
                        temp[j] = node
            res = []
            for node in temp:
                if node != []:
                    t = [head.name, tail.name, node.name]
                    res += [t] 
                else:
                    res += [[]]
        return res, ordinary_dict

# 构建成都的图

In [0]:
#---------------co_map的图
graph = Graph()
flag = 0
l = len(co_map_filter)
for pair in co_map_filter:
    flag += 1
    bid1 = pair.split('#')[0]
    bid2 = pair.split('#')[1]
    val = co_map_filter[pair].strip('\n')
    head = graph.get_node(bid1, create=True)
    tail = graph.get_node(bid2, create=True)
    relation = val
    head.add_edge(Edge(head, tail, relation))
    tail.add_edge(Edge(tail, head, relation))
    if flag % 1000 == 0:
        print '%d / %d\r' %(flag, l),

In [0]:
sfe = SFE(graph)

In [0]:
ba = set()
for pair in co_map_filter:
    bid1 = pair.split('#')[0]
    bid2 = pair.split('#')[1]
    ba.add(bid1)
    ba.add(bid2)
print len(ba)

484903


# 数据构造

In [0]:
def get_tag1(bid):
    tag = poi_dict[bid]['std_tag']
    tag1 = tag.split(';')[0]
    return tag1
def get_tag2(bid):
    tag = poi_dict[bid]['std_tag']
    tag2 = tag.split(';')[-1]
    return tag2
tag1_set = set()
tag2_set = set()
for bid in poi_dict:
    tag1_set.add(get_tag1(bid))
    tag2_set.add(get_tag2(bid))
print len(tag1_set), len(tag2_set)

26 182


In [0]:
def bid_dis(bid1, bid2):
    if bid1 == bid2:
        return 0
    p1 = poi_dict[bid1]['loc']
    p2 = poi_dict[bid2]['loc']
    pp1 = [float(x) for x in p1]
    pp2 = [float(x) for x in p2]
    point1 = Point(pp1)
    point2 = Point(pp2)
    l = point1.distance(point2)
    return l
def bid_co(bid1, bid2):
    if bid1 == bid2:
        return 0
    pair = bid1 + '#' + bid2
    pair1 = bid2 + '#' + bid1
    if pair in co_map_filter:
        val = co_map_filter[pair].strip('\n')
        val = int(val)
    elif pair1 in co_map_filter:
        val = co_map_filter[pair1].strip('\n')
        val = int(val)
    else:
        val = 0
    return val

# motif1

In [0]:
import time
import pickle
st = time.time()
with open('/home/bil/huangtao/project/data/competition/data_75/share_visit.pickle') as f:
    share_visit_res=pickle.load(f)
print time.time() - st
print len(share_visit_res)
import time
st = time.time()
with open('/home/bil/huangtao/project/data/competition/data_75/negtive.pickle') as f:
    negtive=pickle.load(f)
print time.time() - st
print len(negtive)

0.070513010025
8970
0.0395030975342
7423


In [0]:
tag_num_dict = {}
flag = 0
for pair in share_visit_res.keys() + negtive:
    flag+=1
    bid1 = pair.split('#')[0]
    bid2 = pair.split('#')[1]
    try:
        motifs = sfe.sample_motif1(bid1, bid2, 500)
    except:
        continue
    for motif in motifs:
        if motif == []:continue
        tag1 = get_tag1(motif[0])
        tag2 = get_tag1(motif[-1])
        key = tag1 + '#' + tag2
        if key in tag_num_dict:
            tag_num_dict[key] += 1
        else:
            tag_num_dict[key] = 1
    if flag % 100 == 0:
        print '%d / %d\r' %(flag, len(share_visit_res.keys() + negtive)),

In [0]:
motif1_sort = sorted(tag_num_dict.iteritems(),key = lambda x:x[1], reverse=True)
ordinary_list = []
#for i in motif1_sort:
#    print i[0],i[1]
ordinary_list = []
for i in motif1_sort:
    ordinary_list += [i[0]]
    if len(ordinary_list) == 100:break
#for i in ordinary_list:print i
tag_key = ordinary_list[:5]
for i in tag_key:print i

酒店#酒店
酒店#美食
酒店#房地产
酒店#购物
酒店#休闲娱乐


In [0]:
flag = 0
motif_dict = {}
ordinary_dict = {}
for pair in share_visit_res:
    flag+=1
    bid1 = pair.split('#')[0]
    bid2 = pair.split('#')[1]
    temp_dict = {}
    for i in ordinary_list:
        temp_dict[i] = 0
    try:
        motifs, temp_dict = sfe.get_ordinary_motif1(bid1, bid2, temp_dict, 1000, tag_key)#tag_key个motif，搜索m次
    except:
        motifs = [[]] * 5
    motif_dict[pair] = motifs
    ordinary_dict[pair] = temp_dict
    if flag % 100 == 0:
        print '%d / %d\r' %(flag, len(share_visit_res)),

In [0]:
motif_dict_res = {}
flag = 0
for pair in motif_dict:
    flag += 1
    motifs = motif_dict[pair]
    
    temp_dict = ordinary_dict[pair]
    pre = []
    for p in ordinary_list:
        pre += [temp_dict[p]]
    
    for motif in motifs:
        if motif == []:
            if pair in motif_dict_res:
                motif_dict_res[pair] += [[motif, pre, [], []]]
            else:
                motif_dict_res[pair] = [[motif, pre, [], []]]
        else:
            co_sq = [bid_co(motif[0], motif[1]), bid_co(motif[1], motif[2]), bid_co(motif[2], motif[0])]
            dis_sq = [bid_dis(motif[0], motif[1]), bid_dis(motif[1], motif[2]), bid_dis(motif[2], motif[0])]
            if pair in motif_dict_res:
                motif_dict_res[pair] += [[motif, pre, co_sq, dis_sq]]
            else:
                motif_dict_res[pair] = [[motif, pre, co_sq, dis_sq]]
    if flag % 100 == 0:
        print '%d / %d\r' %(flag, len(motif_dict)),

In [0]:
flag = 0
motif_neg_dict = {}
ordinary_neg_dict = {}
for pair in negtive:
    flag+=1
    bid1 = pair.split('#')[0]
    bid2 = pair.split('#')[1]
    temp_dict = {}
    for i in ordinary_list:
        temp_dict[i] = 0
    try:
        motifs, temp_dict = sfe.get_ordinary_motif1(bid1, bid2, temp_dict, 1000, tag_key)#n个motif，搜索m次
    except:
        motifs = [[]] * 5
    motif_neg_dict[pair] = motifs
    ordinary_neg_dict[pair] = temp_dict
    if flag % 100 == 0:
        print '%d / %d\r' %(flag, len(negtive)),

In [0]:
motif_negtive_dict = {}
flag = 0
for pair in motif_neg_dict:
    flag += 1
    motifs = motif_neg_dict[pair]
    
    temp_dict = ordinary_neg_dict[pair]
    pre = []
    for p in ordinary_list:
        pre += [temp_dict[p]]

    for motif in motifs:
        if motif == []:
            if pair in motif_negtive_dict:
                motif_negtive_dict[pair] += [[motif, pre, [], []]]
            else:
                motif_negtive_dict[pair] = [[motif, pre, [], []]]
        else:
            co_sq = [bid_co(motif[0], motif[1]), bid_co(motif[1], motif[2]), bid_co(motif[2], motif[0])]
            dis_sq = [bid_dis(motif[0], motif[1]), bid_dis(motif[1], motif[2]), bid_dis(motif[2], motif[0])]
            if pair in motif_negtive_dict:
                motif_negtive_dict[pair] += [[motif, pre, co_sq, dis_sq]]
            else:
                motif_negtive_dict[pair] = [[motif, pre, co_sq, dis_sq]]
    if flag % 100 == 0:
        print '%d / %d\r' %(flag, len(motif_neg_dict)),

In [0]:
print len(motif_negtive_dict), len(negtive), len(motif_dict_res), len(share_visit_res)
i = 0
for j in motif_dict_res:
    if motif_dict_res[j][0][0] != []:
        i+=1
print i
i = 0
for j in motif_negtive_dict:
    if motif_negtive_dict[j][0][0] != []:
        i+=1
print i
print len(motif_dict_res[motif_dict_res.keys()[0]][0][1])

7423 7423 8970 8970
3288
2252
100


In [0]:
with open('/home/bil/huangtao/project/data/competition/data_75/motif1_dict.pickle', 'wb') as f:
    pickle.dump(motif_dict_res, f)
with open('/home/bil/huangtao/project/data/competition/data_75/motif1_negtive_dict.pickle', 'wb') as f:
    pickle.dump(motif_negtive_dict, f)

# motif2

In [0]:
tag_num_dict = {}
flag = 0
print 'sample...'
for pair in share_visit_res.keys() + negtive:
    flag+=1
    bid1 = pair.split('#')[0]
    bid2 = pair.split('#')[1]
    try:
        motifs = sfe.sample_motif2(bid1, bid2, 500)
    except:
        continue
    for motif in motifs:
        if motif == []:continue
        tag1 = get_tag1(motif[0])
        tag2 = get_tag1(motif[-1])
        key = tag1 + '#' + tag2
        if key in tag_num_dict:
            tag_num_dict[key] += 1
        else:
            tag_num_dict[key] = 1
    if flag % 100 == 0:
        print '%d / %d\r' %(flag, len(share_visit_res.keys() + negtive)),

motif1_sort = sorted(tag_num_dict.iteritems(),key = lambda x:x[1], reverse=True)
ordinary_list = []
#for i in motif1_sort:
#    print i[0],i[1]
ordinary_list = []
for i in motif1_sort:
    ordinary_list += [i[0]]
    if len(ordinary_list) == 100:break
#for i in ordinary_list:print i
tag_key = ordinary_list[:5]
for i in tag_key:print i

sample...
酒店#酒店 / 16393
酒店#美食
酒店#房地产
酒店#购物
酒店#休闲娱乐


In [0]:
flag = 0
motif_dict = {}
ordinary_dict = {}
print 'get_ordinary...'
for pair in share_visit_res:
    flag+=1
    bid1 = pair.split('#')[0]
    bid2 = pair.split('#')[1]
    temp_dict = {}
    for i in ordinary_list:
        temp_dict[i] = 0
    try:
        motifs, temp_dict = sfe.get_ordinary_motif2(bid1, bid2, temp_dict, 1000, tag_key)#tag_key个motif，搜索m次
    except:
        motifs = [[]] * 5
    motif_dict[pair] = motifs
    ordinary_dict[pair] = temp_dict
    if flag % 100 == 0:
        print '%d / %d\r' %(flag, len(share_visit_res)),
motif_dict_res = {}
flag = 0
print 'make data..'
for pair in motif_dict:
    flag += 1
    motifs = motif_dict[pair]
    
    temp_dict = ordinary_dict[pair]
    pre = []
    for p in ordinary_list:
        pre += [temp_dict[p]]
    
    for motif in motifs:
        if motif == []:
            if pair in motif_dict_res:
                motif_dict_res[pair] += [[motif, pre, [], []]]
            else:
                motif_dict_res[pair] = [[motif, pre, [], []]]
        else:
            co_sq = [bid_co(motif[0], motif[1]), bid_co(motif[1], motif[2]), bid_co(motif[2], motif[0])]
            dis_sq = [bid_dis(motif[0], motif[1]), bid_dis(motif[1], motif[2]), bid_dis(motif[2], motif[0])]
            if pair in motif_dict_res:
                motif_dict_res[pair] += [[motif, pre, co_sq, dis_sq]]
            else:
                motif_dict_res[pair] = [[motif, pre, co_sq, dis_sq]]
    if flag % 100 == 0:
        print '%d / %d\r' %(flag, len(motif_dict)),

get_ordinary...
make data..


In [0]:
flag = 0
motif_neg_dict = {}
ordinary_neg_dict = {}
print 'get ordinary...'
for pair in negtive:
    flag+=1
    bid1 = pair.split('#')[0]
    bid2 = pair.split('#')[1]
    temp_dict = {}
    for i in ordinary_list:
        temp_dict[i] = 0
    try:
        motifs, temp_dict = sfe.get_ordinary_motif2(bid1, bid2, temp_dict, 1000, tag_key)#tag_key个motif，搜索m次
    except:
        motifs = [[]] * 5
    motif_neg_dict[pair] = motifs
    ordinary_neg_dict[pair] = temp_dict
    if flag % 100 == 0:
        print '%d / %d\r' %(flag, len(negtive)),
motif_negtive_dict = {}
flag = 0
print 'make data...'
for pair in motif_neg_dict:
    flag += 1
    motifs = motif_neg_dict[pair]
    
    temp_dict = ordinary_neg_dict[pair]
    pre = []
    for p in ordinary_list:
        pre += [temp_dict[p]]

    for motif in motifs:
        if motif == []:
            if pair in motif_negtive_dict:
                motif_negtive_dict[pair] += [[motif, pre, [], []]]
            else:
                motif_negtive_dict[pair] = [[motif, pre, [], []]]
        else:
            co_sq = [bid_co(motif[0], motif[1]), bid_co(motif[1], motif[2]), bid_co(motif[2], motif[0])]
            dis_sq = [bid_dis(motif[0], motif[1]), bid_dis(motif[1], motif[2]), bid_dis(motif[2], motif[0])]
            if pair in motif_negtive_dict:
                motif_negtive_dict[pair] += [[motif, pre, co_sq, dis_sq]]
            else:
                motif_negtive_dict[pair] = [[motif, pre, co_sq, dis_sq]]
    if flag % 100 == 0:
        print '%d / %d\r' %(flag, len(motif_neg_dict)),

get ordinary...
make data...


In [0]:
print len(motif_negtive_dict), len(negtive), len(motif_dict_res), len(share_visit_res)
with open('/home/bil/huangtao/project/data/competition/data_75/motif2_dict.pickle', 'wb') as f:
    pickle.dump(motif_dict_res, f)
with open('/home/bil/huangtao/project/data/competition/data_75/motif2_negtive_dict.pickle', 'wb') as f:
    pickle.dump(motif_negtive_dict, f)

7423 7423 8970 8970


In [0]:
i = 0
for j in motif_dict_res:
    if motif_dict_res[j][0][0] != []:i+=1
print i
i = 0
for j in motif_negtive_dict:
    if motif_negtive_dict[j][0][0] != []:i+=1
print i
print len(motif_dict_res[motif_dict_res.keys()[0]][0][1])

3288
2252
100


In [0]:
tag_num_dict = {}
flag = 0
print 'sample...'
for pair in share_visit_res.keys() + negtive:
    flag+=1
    bid1 = pair.split('#')[0]
    bid2 = pair.split('#')[1]
    try:
        motifs = sfe.sample_motif3(bid1, bid2, 500)
    except:
        continue
    for motif in motifs:
        if motif == []:continue
        tag1 = get_tag1(motif[0])
        tag2 = get_tag1(motif[-1])
        key = tag1 + '#' + tag2
        if key in tag_num_dict:
            tag_num_dict[key] += 1
        else:
            tag_num_dict[key] = 1
    if flag % 100 == 0:
        print '%d / %d\r' %(flag, len(share_visit_res.keys() + negtive)),

motif1_sort = sorted(tag_num_dict.iteritems(),key = lambda x:x[1], reverse=True)
ordinary_list = []
#for i in motif1_sort:
#    print i[0],i[1]
ordinary_list = []
for i in motif1_sort:
    ordinary_list += [i[0]]
    if len(ordinary_list) == 100:break
tag_key = ordinary_list[:5]
for i in tag_key:print i

sample...
酒店#酒店 / 16393
酒店#美食
酒店#旅游景点
美食#美食
酒店#购物


# motif3

In [0]:
flag = 0
motif_dict = {}
ordinary_dict = {}
print 'get_ordinary...'
for pair in share_visit_res:
    flag+=1
    bid1 = pair.split('#')[0]
    bid2 = pair.split('#')[1]
    temp_dict = {}
    for i in ordinary_list:
        temp_dict[i] = 0
    try:
        motifs, temp_dict = sfe.get_ordinary_motif3(bid1, bid2, temp_dict, 1000, tag_key)#n个motif，搜索m次
    except:
        motifs = [[]] * 5
    motif_dict[pair] = motifs
    ordinary_dict[pair] = temp_dict
    if flag % 100 == 0:
        print '%d / %d\r' %(flag, len(share_visit_res)),
motif_dict_res = {}
flag = 0
print 'make data..'
for pair in motif_dict:
    flag += 1
    motifs = motif_dict[pair]
    
    temp_dict = ordinary_dict[pair]
    pre = []
    for p in ordinary_list:
        pre += [temp_dict[p]]
    
    for motif in motifs:
        if motif == []:
            if pair in motif_dict_res:
                motif_dict_res[pair] += [[motif, pre, [], []]]
            else:
                motif_dict_res[pair] = [[motif, pre, [], []]]
        else:
            co_sq = [bid_co(motif[0], motif[1]), bid_co(motif[1], motif[2]), bid_co(motif[2], motif[0])]
            dis_sq = [bid_dis(motif[0], motif[1]), bid_dis(motif[1], motif[2]), bid_dis(motif[2], motif[0])]
            if pair in motif_dict_res:
                motif_dict_res[pair] += [[motif, pre, co_sq, dis_sq]]
            else:
                motif_dict_res[pair] = [[motif, pre, co_sq, dis_sq]]
    if flag % 100 == 0:
        print '%d / %d\r' %(flag, len(motif_dict)),

get_ordinary...
make data..


In [0]:
flag = 0
motif_neg_dict = {}
ordinary_neg_dict = {}
print 'get ordinary...'
for pair in negtive:
    flag+=1
    bid1 = pair.split('#')[0]
    bid2 = pair.split('#')[1]
    temp_dict = {}
    for i in ordinary_list:
        temp_dict[i] = 0
    try:
        motifs, temp_dict = sfe.get_ordinary_motif3(bid1, bid2, temp_dict, 1000, tag_key)#n个motif，搜索m次
    except:
        motifs = [[]] * 5
    motif_neg_dict[pair] = motifs
    ordinary_neg_dict[pair] = temp_dict
    if flag % 100 == 0:
        print '%d / %d\r' %(flag, len(negtive)),
motif_negtive_dict = {}
flag = 0
print 'make data...'
for pair in motif_neg_dict:
    flag += 1
    motifs = motif_neg_dict[pair]
    
    temp_dict = ordinary_neg_dict[pair]
    pre = []
    for p in ordinary_list:
        pre += [temp_dict[p]]

    for motif in motifs:
        if motif == []:
            if pair in motif_negtive_dict:
                motif_negtive_dict[pair] += [[motif, pre, [], []]]
            else:
                motif_negtive_dict[pair] = [[motif, pre, [], []]]
        else:
            co_sq = [bid_co(motif[0], motif[1]), bid_co(motif[1], motif[2]), bid_co(motif[2], motif[0])]
            dis_sq = [bid_dis(motif[0], motif[1]), bid_dis(motif[1], motif[2]), bid_dis(motif[2], motif[0])]
            if pair in motif_negtive_dict:
                motif_negtive_dict[pair] += [[motif, pre, co_sq, dis_sq]]
            else:
                motif_negtive_dict[pair] = [[motif, pre, co_sq, dis_sq]]
    if flag % 100 == 0:
        print '%d / %d\r' %(flag, len(motif_neg_dict)),

get ordinary...
make data...


In [0]:
print len(motif_negtive_dict), len(negtive), len(motif_dict_res), len(share_visit_res)
with open('/home/bil/huangtao/project/data/competition/data_75/motif3_dict.pickle', 'wb') as f:
    pickle.dump(motif_dict_res, f)
with open('/home/bil/huangtao/project/data/competition/data_75/motif3_negtive_dict.pickle', 'wb') as f:
    pickle.dump(motif_negtive_dict, f)

7423 7423 8970 8970


In [0]:
i = 0
for j in motif_dict_res:
    if motif_dict_res[j][0][0] != []:i+=1
print i
i = 0
for j in motif_negtive_dict:
    if motif_negtive_dict[j][0][0] != []:i+=1
print i

658
1692


# motif4

In [0]:
tag_num_dict = {}
flag = 0
print 'sample...'
for pair in share_visit_res.keys() + negtive:
    flag+=1
    bid1 = pair.split('#')[0]
    bid2 = pair.split('#')[1]
    try:
        motifs = sfe.sample_motif4(bid1, bid2, 500)
    except:
        continue
    for motif in motifs:
        if motif == []:continue
        tag1 = get_tag1(motif[0])
        tag2 = get_tag1(motif[-1])
        key = tag1 + '#' + tag2
        if key in tag_num_dict:
            tag_num_dict[key] += 1
        else:
            tag_num_dict[key] = 1
    if flag % 100 == 0:
        print '%d / %d\r' %(flag, len(share_visit_res.keys() + negtive)),

motif1_sort = sorted(tag_num_dict.iteritems(),key = lambda x:x[1], reverse=True)
ordinary_list = []
#for i in motif1_sort:
#    print i[0],i[1]
ordinary_list = []
for i in motif1_sort:
    ordinary_list += [i[0]]
    if len(ordinary_list) == 100:break

tag_key = ordinary_list[:5]
for i in tag_key:print i

sample...
酒店#酒店 / 16393
酒店#美食
酒店#购物
酒店#旅游景点
交通设施#交通设施


In [0]:
flag = 0
motif_dict = {}
ordinary_dict = {}
print 'get_ordinary...'
for pair in share_visit_res:
    flag+=1
    bid1 = pair.split('#')[0]
    bid2 = pair.split('#')[1]
    temp_dict = {}
    for i in ordinary_list:
        temp_dict[i] = 0
    try:
        motifs, temp_dict = sfe.get_ordinary_motif4(bid1, bid2, temp_dict, 1000, tag_key)#n个motif，搜索m次
    except:
        motifs = [[]] * 5
    motif_dict[pair] = motifs
    ordinary_dict[pair] = temp_dict
    if flag % 100 == 0:
        print '%d / %d\r' %(flag, len(share_visit_res)),
motif_dict_res = {}
flag = 0
print 'make data..'
for pair in motif_dict:
    flag += 1
    motifs = motif_dict[pair]
    
    temp_dict = ordinary_dict[pair]
    pre = []
    for p in ordinary_list:
        pre += [temp_dict[p]]
    
    for motif in motifs:
        if motif == []:
            if pair in motif_dict_res:
                motif_dict_res[pair] += [[motif, pre, [], []]]
            else:
                motif_dict_res[pair] = [[motif, pre, [], []]]
        else:
            co_sq = [bid_co(motif[0], motif[1]), bid_co(motif[1], motif[2]), bid_co(motif[2], motif[0])]
            dis_sq = [bid_dis(motif[0], motif[1]), bid_dis(motif[1], motif[2]), bid_dis(motif[2], motif[0])]
            if pair in motif_dict_res:
                motif_dict_res[pair] += [[motif, pre, co_sq, dis_sq]]
            else:
                motif_dict_res[pair] = [[motif, pre, co_sq, dis_sq]]
    if flag % 100 == 0:
        print '%d / %d\r' %(flag, len(motif_dict)),

get_ordinary...
make data..


In [0]:
flag = 0
motif_neg_dict = {}
ordinary_neg_dict = {}
print 'get ordinary...'
for pair in negtive:
    flag+=1
    bid1 = pair.split('#')[0]
    bid2 = pair.split('#')[1]
    temp_dict = {}
    for i in ordinary_list:
        temp_dict[i] = 0
    try:
        motifs, temp_dict = sfe.get_ordinary_motif4(bid1, bid2, temp_dict, 1000, tag_key)#n个motif，搜索m次
    except:
        motifs = [[]] * 5
    motif_neg_dict[pair] = motifs
    ordinary_neg_dict[pair] = temp_dict
    if flag % 100 == 0:
        print '%d / %d\r' %(flag, len(negtive)),
motif_negtive_dict = {}
flag = 0
print 'make data...'
for pair in motif_neg_dict:
    flag += 1
    motifs = motif_neg_dict[pair]
    
    temp_dict = ordinary_neg_dict[pair]
    pre = []
    for p in ordinary_list:
        pre += [temp_dict[p]]

    for motif in motifs:
        if motif == []:
            if pair in motif_negtive_dict:
                motif_negtive_dict[pair] += [[motif, pre, [], []]]
            else:
                motif_negtive_dict[pair] = [[motif, pre, [], []]]
        else:
            co_sq = [bid_co(motif[0], motif[1]), bid_co(motif[1], motif[2]), bid_co(motif[2], motif[0])]
            dis_sq = [bid_dis(motif[0], motif[1]), bid_dis(motif[1], motif[2]), bid_dis(motif[2], motif[0])]
            if pair in motif_negtive_dict:
                motif_negtive_dict[pair] += [[motif, pre, co_sq, dis_sq]]
            else:
                motif_negtive_dict[pair] = [[motif, pre, co_sq, dis_sq]]
    if flag % 100 == 0:
        print '%d / %d\r' %(flag, len(motif_neg_dict)),

get ordinary...
make data...


In [0]:
print len(motif_negtive_dict), len(negtive), len(motif_dict_res), len(share_visit_res)
with open('/home/bil/huangtao/project/data/competition/data_75/motif4_dict.pickle', 'wb') as f:
    pickle.dump(motif_dict_res, f)
with open('/home/bil/huangtao/project/data/competition/data_75/motif4_negtive_dict.pickle', 'wb') as f:
    pickle.dump(motif_negtive_dict, f)

7423 7423 8970 8970


In [0]:
i = 0
for j in motif_dict_res:
    if motif_dict_res[j][0][0] != []:i+=1
print i
i = 0
for j in motif_negtive_dict:
    if motif_negtive_dict[j][0][0] != []:i+=1
print i

3288
2252


In [0]:
print 'load motif1..'
with open('/home/bil/huangtao/project/data/competition/data_75/motif1_dict.pickle', 'rb') as f:
    motif1_dict = pickle.load(f)
with open('/home/bil/huangtao/project/data/competition/data_75/motif1_negtive_dict.pickle', 'rb') as f:
    motif1_negtive_dict = pickle.load(f)
print 'load motif2..'
with open('/home/bil/huangtao/project/data/competition/data_75/motif2_dict.pickle', 'rb') as f:
    motif2_dict = pickle.load(f)
with open('/home/bil/huangtao/project/data/competition/data_75/motif2_negtive_dict.pickle', 'rb') as f:
    motif2_negtive_dict = pickle.load(f)
print 'load motif3..'
with open('/home/bil/huangtao/project/data/competition/data_75/motif3_dict.pickle', 'rb') as f:
    motif3_dict = pickle.load(f)
with open('/home/bil/huangtao/project/data/competition/data_75/motif3_negtive_dict.pickle', 'rb') as f:
    motif3_negtive_dict = pickle.load(f)
print 'load motif4..'
with open('/home/bil/huangtao/project/data/competition/data_75/motif4_dict.pickle', 'rb') as f:
    motif4_dict = pickle.load(f)
with open('/home/bil/huangtao/project/data/competition/data_75/motif4_negtive_dict.pickle', 'rb') as f:
    motif4_negtive_dict = pickle.load(f)

load motif1..
load motif2..
load motif3..
load motif4..


# 特征工程

In [0]:
import pandas as pd
import pickle
import numpy as np

In [0]:
share_list = [x.split('#') + [share_visit[x]] for x in share_visit]
data = pd.DataFrame(share_list,columns = ['bid1','bid2','share_num'])
data['label'] = 1
negtive = [x.split('#') for x in negtive]
data1= pd.DataFrame(negtive,columns = ['bid1','bid2'])
data1['share_num'] = 0
data1['label'] = 0
data = pd.concat((data,data1))

In [0]:
data = data.reset_index(drop=True)
print data.shape
data.head()

(16393, 4)


,bid1,bid2,share_num,label
0,9772759105251718028,1259777061115920383,9,1
1,17995367402019225599,12945627061231106049,28,1
2,12598311716665542083,15536529571081802012,14,1
3,2340999423850109694,12805461576148996146,3,1
4,1670597578991447673,1229039334682718007,4,1


In [0]:
data_cut = data

# poi-feature

In [0]:
import numpy as np
from shapely.geometry import Point
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

/home/bil/anaconda2/lib/python2.7/site-packages/sklearn/externals/joblib/_multiprocessing_helpers.py:28: UserWarning: [Errno 13] Permission denied.  joblib will operate in serial mode
  warnings.warn('%s.  joblib will operate in serial mode' % (e,))


In [0]:
def get_dis(bid1,bid2):
    p1 = poi_dict[bid1]['loc']
    p2 = poi_dict[bid2]['loc']
    pp1 = [float(x) for x in p1]
    pp2 = [float(x) for x in p2]
    point1 = Point(pp1)
    point2 = Point(pp2)
    l = point1.distance(point2)
    return l
data_cut['dis'] = data_cut.apply(lambda x: get_dis(x['bid1'],x['bid2']),axis = 1)

In [0]:
data_cut['bid1_tag1'] = data_cut.apply(lambda x: poi_dict[x['bid1']]['std_tag'].split(';')[0],axis = 1)
data_cut['bid2_tag1'] = data_cut.apply(lambda x: poi_dict[x['bid2']]['std_tag'].split(';')[0],axis = 1)
data_cut['bid1_tag2'] = data_cut.apply(lambda x: poi_dict[x['bid1']]['std_tag'].split(';')[-1],axis = 1)
data_cut['bid2_tag2'] = data_cut.apply(lambda x: poi_dict[x['bid2']]['std_tag'].split(';')[-1],axis = 1)
data_cut = data_cut[data_cut['bid1_tag2'] != '']
data_cut = data_cut[data_cut['bid2_tag2'] != '']
data_cut = data_cut.reset_index(drop=True)

In [0]:
data_cut['bid1_loc1'] = data_cut.apply(lambda x: float(poi_dict[x['bid1']]['loc'][0]),axis = 1)
data_cut['bid1_loc2'] = data_cut.apply(lambda x: float(poi_dict[x['bid1']]['loc'][1]),axis = 1)
data_cut['bid2_loc1'] = data_cut.apply(lambda x: float(poi_dict[x['bid2']]['loc'][0]),axis = 1)
data_cut['bid2_loc2'] = data_cut.apply(lambda x: float(poi_dict[x['bid2']]['loc'][1]),axis = 1)
print data_cut.shape
data_cut.head()

(16393, 13)


,bid1,bid2,share_num,label,dis,bid1_tag1,bid2_tag1,bid1_tag2,bid2_tag2,bid1_loc1,bid1_loc2,bid2_loc1,bid2_loc2
0,9772759105251718028,1259777061115920383,9,1,7393.169747,酒店,酒店,快捷酒店,快捷酒店,11587040.36,3567194.83,11590962.69,3560927.91
1,17995367402019225599,12945627061231106049,28,1,3804.694176,交通设施,交通设施,加油加气站,加油加气站,11591396.48,3562364.22,11588976.56,3559428.29
2,12598311716665542083,15536529571081802012,14,1,317.789967,生活服务,生活服务,通讯营业厅,通讯营业厅,11524734.00,3557872.40,11524907.85,3557606.38
3,2340999423850109694,12805461576148996146,3,1,1163.831259,酒店,酒店,快捷酒店,快捷酒店,11584208.24,3579676.25,11583747.83,3580745.14
4,1670597578991447673,1229039334682718007,4,1,342.126290,生活服务,生活服务,通讯营业厅,通讯营业厅,11581424.68,3569409.06,11581225.53,3569687.25


# 特征构造(region_feature)

In [0]:
import pickle
with open('/home/bil/huangtao/project/data/competition/data_75/bid_town_dict.pickle','rb') as f:
    bid_town_dict = pickle.load(f)

In [0]:
bid1_region = []
for bid in data_cut['bid1']:
    if bid not in bid_town_dict:
        bid1_region +=[dict()]
        continue
    region_ids = bid_town_dict[bid]
    region_id = region_ids[0]
    feature = region_feature[region_id]
    if len(region_ids) != 1:
        region_id = region_ids[1]
        feature1 = region_feature[region_id]
        keys = list(set(feature.keys() + feature1.keys()))
        for key in keys:
            if key in feature and key in feature1:
                feature[key] = (float(feature[key]) + float(feature1[key]))/2.0
            elif key in feature:
                feature[key] = feature[key]/2.0
            elif key in feature1:    
                feature[key] = feature1[key]/2.0
    bid1_region += [feature]

In [0]:
bid2_region = []
for bid in data_cut['bid2']:
    if bid not in bid_town_dict:
        bid2_region +=[dict()]
        continue
    region_ids = bid_town_dict[bid]
    region_id = region_ids[0]
    feature = region_feature[region_id]
    if len(region_ids) != 1:
        region_id = region_ids[1]
        feature1 = region_feature[region_id]
        keys = list(set(feature.keys() + feature1.keys()))
        for key in keys:
            if key in feature and key in feature1:
                feature[key] = (float(feature[key]) + float(feature1[key]))/2.0
            elif key in feature:
                feature[key] = feature[key]/2.0
            elif key in feature1:    
                feature[key] = feature1[key]/2.0
    bid2_region += [feature]

In [0]:
region1 = pd.DataFrame(bid1_region).fillna(0.0)
region2 = pd.DataFrame(bid2_region).fillna(0.0)

In [0]:
region2.head()

,atm_num,c,ktv_num,poi_num,s,专科医院_num,中学_num,中餐厅_num,丽人_num,乡道_num,...,门_num,门址_num,门址点_num,集市_num,风景区_num,飞机场_num,高等院校_num,高速公路_num,高速公路入口_num,高速公路出口_num
0,11.0,0.07044013854,9.0,2120,0.00025341816,0.0,5.0,131.0,73.0,0.0,...,166.0,3.0,3.0,0.0,0.0,0.0,4.0,1.0,0.0,0.0
1,61.0,0.20879735775,7.0,8694,0.00127942123,2.0,6.0,500.0,218.0,0.0,...,642.0,62.0,62.0,0.0,0.0,0.0,7.0,0.0,7.0,7.0
2,58.0,0.50596265441,22.0,9559,0.01156428888,5.0,7.0,556.0,305.0,0.0,...,611.0,13.0,13.0,2.0,4.0,0.0,3.0,1.0,3.0,3.0
3,21.0,0.29443635129,12.0,9556,0.00169908526,0.0,6.0,465.0,288.0,0.0,...,511.0,73.0,73.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
4,150.0,0.20404577778,36.0,20445,0.00093361633,11.0,11.0,1099.0,607.0,0.0,...,1327.0,227.0,227.0,2.0,5.0,0.0,12.0,2.0,5.0,3.0


In [0]:
region1_scaled = preprocessing.scale(region1)
region2_scaled = preprocessing.scale(region2)

In [0]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
def labels_to_onehot(label):
    enc_label = LabelEncoder()
    enc_label.fit(label)
    encode = enc_label.transform(label)
    encode = np.array([encode]).T
    enc_onehot = OneHotEncoder()
    enc_onehot.fit(encode)
    onehot = enc_onehot.transform(encode)
    return onehot
all_bid = set()
for pair in co_map_filter:
    bid1 = pair.split('#')[0]
    bid2 = pair.split('#')[1]
    all_bid.add(bid1)
    all_bid.add(bid2)
for bid in data_cut['bid1']:
    all_bid.add(bid)
for bid in data_cut['bid2']:
    all_bid.add(bid)   
bid_list = list(all_bid)
tag1_list = []
tag2_list = []
for bid in bid_list:
    tag1 = poi_dict[bid]['std_tag'].split(';')[0]
    tag2 = poi_dict[bid]['std_tag'].split(';')[-1]
    tag1_list += [tag1]
    tag2_list += [tag2]
onehot1 = labels_to_onehot(tag1_list)
onehot2 = labels_to_onehot(tag2_list)
loc1_list = []
loc2_list = []
for bid in bid_list:
    loc1 = poi_dict[bid]['loc'][0]
    loc2 = poi_dict[bid]['loc'][1]
    loc1_list += [loc1]
    loc2_list += [loc2]
loc1_scaled = preprocessing.scale(loc1_list)
loc2_scaled = preprocessing.scale(loc2_list)  

In [0]:
all_bid = set()
for pair in co_map_filter:
    bid1 = pair.split('#')[0]
    bid2 = pair.split('#')[1]
    all_bid.add(bid1)
    all_bid.add(bid2)
for bid in data_cut['bid1']:
    all_bid.add(bid)
for bid in data_cut['bid2']:
    all_bid.add(bid)  
all_bid = list(all_bid)
i = 0
poi_index = {}
for bid in all_bid:
    i+=1
    poi_index[bid] = i

In [0]:
poi_embed_dict = {}
flag = 0
for i in range(len(bid_list)):
    flag += 1
    bid = bid_list[i]
    tag1 = onehot1[i]
    tag2 = onehot2[i]
    loc1 = loc1_scaled[i]
    loc2 = loc2_scaled[i]
    poi_embed_dict[bid] = np.array([tag1.toarray()[0].tolist() + tag2.toarray()[0].tolist() + [loc1]+[loc2]])
    if flag % 1000 == 0:print '%d / %d\r' %(flag, len(bid_list)),

In [0]:
bid1_embed = []
for bid in data_cut['bid1']:
    embed = poi_embed_dict[bid][0]
    bid1_embed += [embed]
bid2_embed = []
for bid in data_cut['bid2']:
    embed = poi_embed_dict[bid][0]
    bid2_embed += [embed]
bid1_embed = np.array(bid1_embed)
bid2_embed = np.array(bid2_embed)
print bid1_embed.shape, bid2_embed.shape

(16393, 209) (16393, 209)


In [0]:
label = np.array(data_cut.loc[:,'label'])
all_index = range(len(data))
from sklearn.model_selection import train_test_split
train_index,test_index = train_test_split(all_index,test_size=0.1,random_state=2)

In [0]:
from sklearn import preprocessing
dis_scaled = preprocessing.scale(data_cut.loc[:,'dis'])
co_map_list = []
for i in range(len(data_cut)):
    flag +=1
    res = []
    bid1 = data_cut.loc[i,'bid1']
    bid2 = data_cut.loc[i,'bid2']
    pair = bid1 + '#' + bid2
    pair1 = bid2 + '#' + bid1
    if pair in co_map_filter:
        val = int(co_map_filter[pair].strip('\n'))
    elif pair1 in co_map_filter:
        val = int(co_map_filter[pair1].strip('\n'))
    else:
        val = 0
    co_map_list += [val]
co_map_scaled = preprocessing.scale(co_map_list)
co_feature = np.concatenate([np.array([dis_scaled]).T, 
                         np.array([co_map_scaled]).T],axis = 1)
print co_feature.shape

(16393, 2)


# 保存数据

In [0]:
import pickle
with open('/home/bil/huangtao/project/data/competition/data_75/data_to_gpu/co_feature.pickle','wb') as f:
    pickle.dump(co_feature,f)
with open('/home/bil/huangtao/project/data/competition/data_75/data_to_gpu/bid1_embed.pickle','wb') as f:
    pickle.dump(bid1_embed,f)
with open('/home/bil/huangtao/project/data/competition/data_75/data_to_gpu/bid2_embed.pickle','wb') as f:
    pickle.dump(bid2_embed,f) 
with open('/home/bil/huangtao/project/data/competition/data_75/data_to_gpu/region1_embed.pickle','wb') as f:
    pickle.dump(region1_scaled,f) 
with open('/home/bil/huangtao/project/data/competition/data_75/data_to_gpu/region2_embed.pickle','wb') as f:
    pickle.dump(region2_scaled,f)
with open('/home/bil/huangtao/project/data/competition/data_75/data_to_gpu/train_index.pickle','wb') as f:
    pickle.dump(train_index,f)
with open('/home/bil/huangtao/project/data/competition/data_75/data_to_gpu/test_index.pickle','wb') as f:
    pickle.dump(test_index,f)
with open('/home/bil/huangtao/project/data/competition/data_75/data_to_gpu/label.pickle','wb') as f:
    pickle.dump(np.array(data_cut.loc[:,'label']),f)

In [0]:
print co_feature.shape,bid1_embed.shape,bid2_embed.shape,region1_scaled.shape,region2_scaled.shape

(16393, 2) (16393, 209) (16393, 209) (16393, 185) (16393, 185)


In [0]:
import pickle
with open('/home/bil/huangtao/project/data/competition/data_75/data_to_gpu/co_feature.pickle','rb') as f:
    co_feature = pickle.load(f)
with open('/home/bil/huangtao/project/data/competition/data_75/data_to_gpu/bid1_embed.pickle','rb') as f:
    bid1_embed = pickle.load(f)
with open('/home/bil/huangtao/project/data/competition/data_75/data_to_gpu/bid2_embed.pickle','rb') as f:
    bid2_embed = pickle.load(f)
with open('/home/bil/huangtao/project/data/competition/data_75/data_to_gpu/region1_embed.pickle','rb') as f:
    region1 = pickle.load(f)
with open('/home/bil/huangtao/project/data/competition/data_75/data_to_gpu/region2_embed.pickle','rb') as f:
    region2 = pickle.load(f)
with open('/home/bil/huangtao/project/data/competition/data_75/data_to_gpu/train_index.pickle','rb') as f:
    train_index = pickle.load(f)
with open('/home/bil/huangtao/project/data/competition/data_75/data_to_gpu/test_index.pickle','rb') as f:
    test_index = pickle.load(f)
with open('/home/bil/huangtao/project/data/competition/data_75/data_to_gpu/label.pickle','rb') as f:
    label = pickle.load(f)

In [0]:
from sklearn import metrics
def change_score(score, label):
    max_score = max(score)
    predict = [float(x)/max_score for x in score]
    p = predict
    predict = [int(x + 0.5) for x in predict]
    acc = metrics.accuracy_score(predict,label)
    max_acc = 0
    max_score = max(score)
    predict = [float(x)/max_score for x in score]
    res = []
    for i in range(20,80):
        j = float(i)/100
        t = [int(x + j) for x in p]
        acc1 = metrics.accuracy_score(t,label)
        if acc1 > max_acc:
            max_acc = acc1
            pp = t
    print 'acc:',acc,'max_acc:',max_acc
    return p,pp
def get_score(t, p, pp):
    acc = metrics.accuracy_score(t,pp)
    f1 = metrics.f1_score(t,pp)
    auc = metrics.roc_auc_score(t,p)
    precision = metrics.precision_score(t,pp)
    recall = metrics.recall_score(t,pp)
    print 'acc: ',acc,'auc: ',auc,'f1_score: ',f1,'\nprecision: ',precision, 'recall: ', recall
    return

# model1:co-map_val

In [0]:
score = []
for i in range(len(data)):
    res = []
    bid1 = data.loc[i,'bid1']
    bid2 = data.loc[i,'bid2']
    pair = bid1 + '#' + bid2
    pair1 = bid2 + '#' + bid1
    if pair in co_map_filter:
        val = int(co_map_filter[pair].strip('\n'))
    elif pair1 in co_map_filter:
        val = int(co_map_filter[pair1].strip('\n'))
    else:
        val = 0
    score += [val]

In [0]:
p,pp = change_score(score, label)

acc: 0.4532422375404136 max_acc: 0.45714634295125967


In [0]:
l = [1-x for x in label]
p,pp = change_score(score, l)
get_score(l,p,pp)

acc: 0.5467577624595864 max_acc: 0.5470627706948088
acc:  0.5470627706948088 auc:  0.31529662168159434 f1_score:  0.0 
precision:  0.0 recall:  0.0


# pa

In [0]:
score = []
for i in range(len(data)):
    val = 1.0
    bid1 = data.loc[i,'bid1']
    bid2 = data.loc[i,'bid2']
    try:
        head = graph.nodes[bid1]
        head_nears = head.neighbor2edgesstr
        for head_near in head_nears:
            val *= float(head_nears[head_near][0]) /10000.0
    except:
        val = 1.0
    try:
        tail = graph.nodes[bid2]
        tail_nears = tail.neighbor2edgesstr
        for tail_near in tail_nears:
            val *= float(tail_nears[tail_near][0]) / 10000.0
    except:
        pass
    if bid1 not in graph.nodes and bid2 not in graph.nodes:
        val = 0.0
    score += [val]
    if i % 100 == 0:print '\r%d / %d' %(i, len(data)),

16300 / 16393                                                                                              


In [0]:
p,pp = change_score(score, label)
l = [1-x for x in label]
p,pp = change_score(score, l)
get_score(l,p,pp)

acc: 0.45293722930519126 max_acc: 0.45293722930519126
acc: 0.5470627706948088 max_acc: 0.5470627706948088
acc:  0.5470627706948088 auc:  0.4894425653731338 f1_score:  0.0 
precision:  0.0 recall:  0.0


# CN

In [0]:
score = []
for i in range(len(data)):
    val = 0
    bid1 = data.loc[i,'bid1']
    bid2 = data.loc[i,'bid2']
    try:
        head = graph.nodes[bid1]
        tail = graph.nodes[bid2]
    except:
        val = 0
        score += [val]
        continue
    head_nears = tail.neighbor2edgesstr
    tail_nears = tail.neighbor2edgesstr
    for head_near in head_nears:
        if head_near in tail_nears:
            val += int(head_nears[head_near][0])
    for tail_near in tail_nears:
        if tail_near in head_nears:
            val += int(tail_nears[tail_near][0])
    score += [val]
    if i % 100 == 0:print '\r%d / %d' %(i, len(data)),

16300 / 16393                                                                                                      


In [0]:
p,pp = change_score(score, label)
l = [1-x for x in label]
p,pp = change_score(score, l)
get_score(l,p,pp)

acc: 0.45787836271579335 max_acc: 0.4769718782407125
acc: 0.5421216372842067 max_acc: 0.546452754224364
acc:  0.546452754224364 auc:  0.4796969961842362 f1_score:  0.0018794469056249163 
precision:  0.2692307692307692 recall:  0.0009430149535228344


# RA

In [0]:
score = []
for i in range(200):
    val = 0
    bid1 = data.loc[i,'bid1']
    bid2 = data.loc[i,'bid2']
    try:
        head = graph.nodes[bid1]
        tail = graph.nodes[bid2]
    except:
        val = 0
        score += [val]
        continue
    head_nears = tail.neighbor2edgesstr
    tail_nears = tail.neighbor2edgesstr
    for head_near in head_nears:
        if head_near in tail_nears:
            near_sum = int(head_nears[head_near][0]) + int(tail_nears[head_near][0])
            ex_sum = 0
            head_near_nears = head_near.neighbor2edgesstr
            for head_near_near in head_near_nears:
                ex_sum += int(head_near_nears[head_near_near][0])
            val += float(near_sum) / ex_sum
    score += [val]
    if i % 1 == 0:print '\r%d / %d' %(i, len(data)),

for i in range(len(data)-200,len(data)):
    val = 0
    bid1 = data.loc[i,'bid1']
    bid2 = data.loc[i,'bid2']
    try:
        head = graph.nodes[bid1]
        tail = graph.nodes[bid2]
    except:
        val = 0
        score += [val]
        continue
    head_nears = tail.neighbor2edgesstr
    tail_nears = tail.neighbor2edgesstr
    for head_near in head_nears:
        if head_near in tail_nears:
            near_sum = int(head_nears[head_near][0]) + int(tail_nears[head_near][0])
            ex_sum = 0
            head_near_nears = head_near.neighbor2edgesstr
            for head_near_near in head_near_nears:
                ex_sum += int(head_near_nears[head_near_near][0])
            val += float(near_sum) / ex_sum
    score += [val]
    if i % 1 == 0:print '\r%d / %d' %(i, len(data)),

16391 / 16393                                                                                                                                                                                                                                                                                                                      


In [0]:
ls = [1]*200 + [0]*200
p,pp = change_score(score, ls)
l = [1-x for x in ls]
p,pp = change_score(score, l)
get_score(l,p,pp)

acc: 0.4975 max_acc: 0.54
acc: 0.5025 max_acc: 0.515
acc:  0.515 auc:  0.42525 f1_score:  0.07619047619047618 
precision:  0.8 recall:  0.04


# JACC

In [0]:
score = []
for i in range(1000):
    val = 0
    bid1 = data.loc[i,'bid1']
    bid2 = data.loc[i,'bid2']
    try:
        head = graph.nodes[bid1]
        tail = graph.nodes[bid2]
    except:
        val = 0
        score += [val]
        continue
    head_nears = tail.neighbor2edgesstr
    tail_nears = tail.neighbor2edgesstr
    ex_sum = 0
    for head_near in head_nears:
        ex_sum += int(head_nears[head_near][0])
    for tail_near in tail_nears:
        ex_sum += int(tail_nears[tail_near][0])
        
    for head_near in head_nears:
        if head_near in tail_nears:
            near_sum = int(head_nears[head_near][0]) + int(tail_nears[head_near][0])
            val += float(near_sum) / ex_sum
    score += [val]
    if i % 1 == 0:print '\r%d / %d' %(i, len(data)),

for i in range(len(data)-1000,len(data)):
    val = 0
    bid1 = data.loc[i,'bid1']
    bid2 = data.loc[i,'bid2']
    try:
        head = graph.nodes[bid1]
        tail = graph.nodes[bid2]
    except:
        val = 0
        score += [val]
        continue
    head_nears = tail.neighbor2edgesstr
    tail_nears = tail.neighbor2edgesstr
    ex_sum = 0
    for head_near in head_nears:
        ex_sum += int(head_nears[head_near][0])
    for tail_near in tail_nears:
        ex_sum += int(tail_nears[tail_near][0])
        
    for head_near in head_nears:
        if head_near in tail_nears:
            near_sum = int(head_nears[head_near][0]) + int(tail_nears[head_near][0])
            val += float(near_sum) / ex_sum
    score += [val]
    if i % 1 == 0:print '\r%d / %d' %(i, len(data)),

16391 / 16393                                                                          


In [0]:
ls = [1]*1000 + [0]*1000
p,pp = change_score(score, ls)
get_score(ls,p,pp)

acc: 0.5 max_acc: 0.5
acc:  0.5 auc:  0.5232970000000001 f1_score:  0.6655518394648829 
precision:  0.5 recall:  0.995


# word2vec

In [0]:
import sys
import argparse
import numpy as np
import networkx as nx
from gensim.models import Word2Vec

In [0]:
val_list = []
for pair in co_map_filter:
    bid1 = pair.split('#')[0]
    bid2 = pair.split('#')[1]
    val = co_map_filter[pair].strip('\n')
    val_list += [int(val)]

In [0]:
float(sum(val_list)) / len(val_list)

39.87339073638693

In [0]:
co_map_nodevec = {}
val_list = []
for pair in co_map_filter:
    bid1 = pair.split('#')[0]
    bid2 = pair.split('#')[1]
    val = co_map_filter[pair].strip('\n')
    if int(val) > 100:
        co_map_nodevec[pair] = co_map_filter[pair]

In [0]:
print len(co_map_nodevec), len(co_map_filter)

532949 10317918


In [0]:
#---------------co_map的图----------
G = nx.Graph()
flag = 0
l = len(co_map_nodevec)
for pair in co_map_nodevec:
    flag += 1
    bid1 = pair.split('#')[0]
    bid2 = pair.split('#')[1]
    val = co_map_filter[pair].strip('\n')
    tag = poi_dict[bid1]['std_tag'].split(';')[0]
    G.add_node(int(bid1))  
    G.add_node(int(bid2))
    G.add_edge(int(bid1),int(bid2),weight=float(val))
    if flag % 1000 == 0:
        print '%d / %d\r' %(flag, l),

In [0]:
import numpy as np
import networkx as nx
import random


class node2vecgraph():
    def __init__(self, nx_G, is_directed, p, q):
        self.G = nx_G
        self.is_directed = is_directed
        self.p = p
        self.q = q

    def node2vec_walk(self, walk_length, start_node):
        '''
        Simulate a random walk starting from start node.
        '''
        G = self.G
        alias_nodes = self.alias_nodes
        alias_edges = self.alias_edges

        walk = [start_node]

        while len(walk) < walk_length:
            cur = walk[-1]
            cur_nbrs = sorted(G.neighbors(cur))
            if len(cur_nbrs) > 0:
                if len(walk) == 1:
                    walk.append(cur_nbrs[alias_draw(alias_nodes[cur][0], alias_nodes[cur][1])])
                else:
                    prev = walk[-2]
                    next = cur_nbrs[alias_draw(alias_edges[(prev, cur)][0], 
                        alias_edges[(prev, cur)][1])]
                    walk.append(next)
            else:
                break

        return walk

    def simulate_walks(self, num_walks, walk_length):
        '''
        Repeatedly simulate random walks from each node.
        '''
        G = self.G
        walks = []
        nodes = list(G.nodes())
        print 'Walk iteration:'
        for walk_iter in range(num_walks):
            print str(walk_iter+1), '/', str(num_walks)
            random.shuffle(nodes)
            for node in nodes:
                walks.append(self.node2vec_walk(walk_length=walk_length, start_node=node))

        return walks

    def get_alias_edge(self, src, dst):
        '''
        Get the alias edge setup lists for a given edge.
        '''
        G = self.G
        unnormalized_probs = []
        for dst_nbr in sorted(G.neighbors(dst)):
            unnormalized_probs.append(G[dst][dst_nbr]['weight'])
        norm_const = sum(unnormalized_probs)
        normalized_probs =  [float(u_prob)/norm_const for u_prob in unnormalized_probs]

        return alias_setup(normalized_probs)

    def preprocess_transition_probs(self):
        '''
        Preprocessing of transition probabilities for guiding the random walks.
        '''
        G = self.G
        is_directed = self.is_directed

        alias_nodes = {}
        flag = 0
        l = G.number_of_nodes()
        print 'building...'
        for node in G.nodes():
            flag+=1
            unnormalized_probs = [G[node][nbr]['weight'] for nbr in sorted(G.neighbors(node))]
            norm_const = sum(unnormalized_probs)
            normalized_probs =  [float(u_prob)/norm_const for u_prob in unnormalized_probs]
            alias_nodes[node] = alias_setup(normalized_probs)
            if flag % 10 == 0:
                print '\r%d / %d' %(flag, l),

        alias_edges = {}
        triads = {}
        print '\nworking...'
        if is_directed:
            for edge in G.edges():
                alias_edges[edge] = self.get_alias_edge(edge[0], edge[1])
        else:
            l = G.number_of_edges()
            flag = 0
            for edge in G.edges():
                flag+=1
                alias_edges[edge] = self.get_alias_edge(edge[0], edge[1])
                alias_edges[(edge[1], edge[0])] = self.get_alias_edge(edge[1], edge[0])
                if flag % 100 == 0:
                    print '\r%d / %d' %(flag, l),
        self.alias_nodes = alias_nodes
        self.alias_edges = alias_edges

        return


def alias_setup(probs):
    '''
    Compute utility lists for non-uniform sampling from discrete distributions.
    Refer to https://hips.seas.harvard.edu/blog/2013/03/03/the-alias-method-efficient-sampling-with-many-discrete-outcomes/
    for details
    '''
    K = len(probs)
    q = np.zeros(K)
    J = np.zeros(K, dtype=np.int)

    smaller = []
    larger = []
    for kk, prob in enumerate(probs):
        q[kk] = K*prob
        if q[kk] < 1.0:
            smaller.append(kk)
        else:
            larger.append(kk)

    while len(smaller) > 0 and len(larger) > 0:
        small = smaller.pop()
        large = larger.pop()

        J[small] = large
        q[large] = q[large] + q[small] - 1.0
        if q[large] < 1.0:
            smaller.append(large)
        else:
            larger.append(large)

    return J, q

def alias_draw(J, q):
    '''
    Draw sample from a non-uniform discrete distribution using alias sampling.
    '''
    K = len(J)

    kk = int(np.floor(np.random.rand()*K))
    if np.random.rand() < q[kk]:
        return kk
    else:
        return J[kk]

In [0]:
Gn = node2vecgraph(G, False, 1, 1)
Gn.preprocess_transition_probs()

building...
119790 / 119799                   
working...
532900 / 532949                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               

In [0]:
walks = Gn.simulate_walks(10, 10)

Walk iteration:
1 / 10
2 / 10
3 / 10
4 / 10
5 / 10
6 / 10
7 / 10
8 / 10
9 / 10
10 / 10


In [0]:
def learn_embeddings(walks):
	'''
	Learn embeddings by optimizing the Skipgram objective using SGD.
	'''
	walks = [map(str, walk) for walk in walks]
	model = Word2Vec(walks, size=128, window=10, min_count=0, sg=1, workers=8, iter=1)

	return model
def cosine_similarity(vector1, vector2):
    dot_product = 0.0
    normA = 0.0
    normB = 0.0
    for a, b in zip(vector1, vector2):
        dot_product += a * b
        normA += a ** 2
        normB += b ** 2
    if normA == 0.0 or normB == 0.0:
        return 0
    else:
        return round(dot_product / ((normA**0.5)*(normB**0.5)) * 100, 2)
vector = learn_embeddings(walks)

In [0]:
t = []
p = []
f = []
for i in range(len(data)):
    val = 0
    bid1 = data.loc[i,'bid1']
    bid2 = data.loc[i,'bid2']
    tag1 = poi_dict[bid1]['std_tag'].split(';')[0]
    try:
        m1 = vector[bid1]
    except:
        m1 = [0.0] * 128
    try:
        m2 = vector[bid2]
    except:
        m2 = [0.0] * 128
    cos = cosine_similarity(m1, m2)
    f += [list(m1) + list(m2)]
    p += [cos]
    t += [data.loc[i,'label']]   

/home/bil/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # Remove the CWD from sys.path while we load stuff.
/home/bil/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:14: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [0]:
p,pp = change_score(p, label)
get_score(label,p,pp)

acc: 0.5578600622216799 max_acc: 0.6106264869151468
acc:  0.6106264869151468 auc:  0.6549813462060357 f1_score:  0.6618994650140368 
precision:  0.630537894843072 recall:  0.6965440356744704


In [0]:
train_x = np.array(f)[train_index,:]
train_y = np.array(t)[train_index]
test_x = np.array(f)[test_index,:]
test_y = np.array(t)[test_index]
print np.array(f).shape

(16393, 256)


In [0]:
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
print '逻辑回归模型:\n',model

逻辑回归模型:
LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)


In [0]:
model.fit(train_x,train_y)
score = model.predict_proba(test_x)  #预测分类概率
predict_y = model.predict(test_x)
acc = model.score(test_x,test_y)
pp = [int(x[1]+0.5) for x in score]
get_score(test_y,score[:,1],pp)

acc:  0.6780487804878049 auc:  0.7361802967316513 f1_score:  0.697594501718213 
precision:  0.6967963386727689 recall:  0.698394495412844


# feature-MLP

In [0]:
data_x = np.concatenate([co_feature, bid1_embed, bid2_embed],axis = 1)
print data_x.shape

(16393, 420)


In [0]:
train_y = label[train_index]
test_y = label[test_index]
train_x = data_x[train_index,:]
test_x = data_x[test_index,:]

In [0]:
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
print '逻辑回归模型:\n',model

逻辑回归模型:
LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)


In [0]:
model.fit(train_x,train_y)
score = model.predict_proba(test_x)  #预测分类概率
predict_y = model.predict(test_x)
acc = model.score(test_x,test_y)
pp = [int(x[1]+0.5) for x in score]
get_score(test_y,score[:,1],pp)

acc:  0.6908536585365853 auc:  0.7579782468463303 f1_score:  0.7124220079410096 
precision:  0.7048260381593715 recall:  0.7201834862385321


# node2vec MLP

In [0]:
t = []
p = []
f = []
for i in range(len(data)):
    val = 0
    bid1 = data.loc[i,'bid1']
    bid2 = data.loc[i,'bid2']
    tag1 = poi_dict[bid1]['std_tag'].split(';')[0]
    try:
        m1 = vector[bid1]
    except:
        m1 = [0.0] * 128
    try:
        m2 = vector[bid2]
    except:
        m2 = [0.0] * 128
    cos = cosine_similarity(m1, m2)
    f += [list(m1) + list(m2)]
    p += [cos]
    t += [data.loc[i,'label']]   

/home/bil/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # Remove the CWD from sys.path while we load stuff.
/home/bil/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:14: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [0]:
data_x = np.concatenate([co_feature, bid1_embed,bid2_embed, f],axis = 1)

In [0]:
train_y = np.array(t)[train_index]
test_y = np.array(t)[test_index]
train_x = np.array(data_x)[train_index,:]
test_x = np.array(data_x)[test_index,:]

In [0]:
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
print '逻辑回归模型:\n',model

逻辑回归模型:
LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)


In [0]:
model.fit(train_x,train_y)
score = model.predict_proba(test_x)  #预测分类概率
predict_y = model.predict(test_x)
acc = model.score(test_x,test_y)
pp = [int(x[1]+0.5) for x in score]
get_score(test_y,score[:,1],pp)

acc:  0.7524390243902439 auc:  0.8327853234900612 f1_score:  0.7698412698412699 
precision:  0.7612107623318386 recall:  0.7786697247706422


# XGBoost

In [0]:
data_x = np.concatenate([co_feature, bid1_embed, bid2_embed, region1,region2],axis = 1)
np.array(data_x).shape

(16393, 790)

In [0]:
train_y = label[train_index]
test_y = label[test_index]
train_x = np.array(data_x)[train_index,:]
test_x = np.array(data_x)[test_index,:]

In [0]:
import xgboost as xgb
xgbmodel = xgb.XGBClassifier(n_estimators=100, 
                             silent=True, 
                             objective='binary:logistic',
                             max_depth=2, 
                             learning_rate=0.1)
xgbmodel.fit(train_x,train_y,early_stopping_rounds=10,eval_set=[(test_x,test_y)],eval_metric="error")

[0]	validation_0-error:0.311585
Will train until validation_0-error hasn't improved in 10 rounds.
[1]	validation_0-error:0.311585
[2]	validation_0-error:0.311585
[3]	validation_0-error:0.311585
[4]	validation_0-error:0.292073
[5]	validation_0-error:0.292073
[6]	validation_0-error:0.277439
[7]	validation_0-error:0.287195
[8]	validation_0-error:0.277439
[9]	validation_0-error:0.277439
[10]	validation_0-error:0.277439
[11]	validation_0-error:0.278049
[12]	validation_0-error:0.276829
[13]	validation_0-error:0.276829
[14]	validation_0-error:0.276829
[15]	validation_0-error:0.27622
[16]	validation_0-error:0.27378
[17]	validation_0-error:0.273171
[18]	validation_0-error:0.273171
[19]	validation_0-error:0.271341
[20]	validation_0-error:0.272561
[21]	validation_0-error:0.273171
[22]	validation_0-error:0.271951
[23]	validation_0-error:0.271951
[24]	validation_0-error:0.273171
[25]	validation_0-error:0.272561
[26]	validation_0-error:0.272561
[27]	validation_0-error:0.272561
[28]	validation_0-erro

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=2, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [0]:
score = xgbmodel.predict_proba(test_x)  #预测分类概率
predict_y = xgbmodel.predict(test_x)
pp = [int(x[1]+0.5) for x in score]
get_score(test_y,score[:,1],pp)

acc:  0.7286585365853658 auc:  0.7871220673260704 f1_score:  0.7374631268436577 
precision:  0.7594167679222357 recall:  0.716743119266055


/home/bil/anaconda2/lib/python2.7/site-packages/sklearn/preprocessing/label.py:171: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


# motif

In [0]:
#------------motif1---------------
motif1_feature = []#[poi*3,pre,co_sq, dis_sq]*5
flag = 0
for i in range(len(data_cut)):
    flag +=1
    bid1 = data_cut.loc[i,'bid1']
    bid2 = data_cut.loc[i,'bid2']
    pair = bid1 + '#' + bid2
    if pair in motif1_dict:
        val = motif1_dict[pair]
    else:
        val = motif1_negtive_dict[pair]#[poi*3,pre,co_sq, dis_sq]*5
    embed1 = poi_embed_dict[bid1][0]
    embed2 = poi_embed_dict[bid2][0]
    line = val
    motif1_feature += [line]
    if flag % 100 == 0:print '%d / %d\r' %(flag, len(data_cut)),

In [0]:
#------------motif2---------------
motif2_feature = []#[poi*3,pre,co_sq, dis_sq]*5
flag = 0
for i in range(len(data_cut)):
    flag +=1
    bid1 = data_cut.loc[i,'bid1']
    bid2 = data_cut.loc[i,'bid2']
    pair = bid1 + '#' + bid2
    if pair in motif2_dict:
        val = motif2_dict[pair]
    else:
        val = motif2_negtive_dict[pair]#[poi*3,pre,co_sq, dis_sq]*5
    line = val
    motif2_feature += [line]
    if flag % 100 == 0:print '%d / %d\r' %(flag, len(data_cut)),

In [0]:
#------------motif3---------------
motif3_feature = []#[poi*3,pre,co_sq, dis_sq]*5
flag = 0
for i in range(len(data_cut)):
    flag +=1
    bid1 = data_cut.loc[i,'bid1']
    bid2 = data_cut.loc[i,'bid2']
    pair = bid1 + '#' + bid2
    if pair in motif3_dict:
        val = motif3_dict[pair]
    else:
        val = motif3_negtive_dict[pair]#[poi*3,pre,co_sq, dis_sq]*5
    line = val
    motif3_feature += [line]
    if flag % 100 == 0:print '%d / %d\r' %(flag, len(data_cut)),

In [0]:
#------------motif4---------------
motif4_feature = []#[poi*3,pre,co_sq, dis_sq]*5
flag = 0
for i in range(len(data_cut)):
    flag +=1
    bid1 = data_cut.loc[i,'bid1']
    bid2 = data_cut.loc[i,'bid2']
    pair = bid1 + '#' + bid2
    if pair in motif4_dict:
        val = motif4_dict[pair]
    else:
        val = motif4_negtive_dict[pair]#[poi*3,pre,co_sq, dis_sq]*5
    line = val
    motif4_feature += [line]
    if flag % 100 == 0:print '%d / %d\r' %(flag, len(data_cut)),

In [0]:
poi_embed_dict_filter = {}
print 'add 1...'
for val in motif1_feature:
    for v in val:
        motif = v[0]
        if motif == []:continue
        poi_embed_dict_filter[motif[0]] = poi_embed_dict[motif[0]][0]
        poi_embed_dict_filter[motif[1]] = poi_embed_dict[motif[1]][0]
        poi_embed_dict_filter[motif[2]] = poi_embed_dict[motif[2]][0]
print 'add 2...'
for val in motif2_feature:
    for v in val:
        motif = v[0]
        if motif == []:continue
        poi_embed_dict_filter[motif[0]] = poi_embed_dict[motif[0]][0]
        poi_embed_dict_filter[motif[1]] = poi_embed_dict[motif[1]][0]
        poi_embed_dict_filter[motif[2]] = poi_embed_dict[motif[2]][0]
print 'add 3...'
for val in motif3_feature:
    for v in val:
        motif = v[0]
        if motif == []:continue
        poi_embed_dict_filter[motif[0]] = poi_embed_dict[motif[0]][0]
        poi_embed_dict_filter[motif[1]] = poi_embed_dict[motif[1]][0]
        poi_embed_dict_filter[motif[2]] = poi_embed_dict[motif[2]][0]
print 'add 4...'
for val in motif4_feature:
    for v in val:
        motif = v[0]
        if motif == []:continue
        poi_embed_dict_filter[motif[0]] = poi_embed_dict[motif[0]][0]
        poi_embed_dict_filter[motif[1]] = poi_embed_dict[motif[1]][0]
        poi_embed_dict_filter[motif[2]] = poi_embed_dict[motif[2]][0]

add 1...
add 2...
add 3...
add 4...


In [0]:
print 'save...'
with open('/home/bil/huangtao/project/data/competition/data_75/data_to_gpu/poi_embed_dict.pickle','wb') as f:
    pickle.dump(poi_embed_dict_filter, f)
print 'save...'
with open('/home/bil/huangtao/project/data/competition/data_75/data_to_gpu/motif1_feature.pickle','wb') as f:
    pickle.dump(motif1_feature,f)
print 'save...'
with open('/home/bil/huangtao/project/data/competition/data_75/data_to_gpu/motif2_feature.pickle','wb') as f:
    pickle.dump(motif2_feature,f)
print 'save...'
with open('/home/bil/huangtao/project/data/competition/data_75/data_to_gpu/motif3_feature.pickle','wb') as f:
    pickle.dump(motif3_feature,f)
print 'save...'
with open('/home/bil/huangtao/project/data/competition/data_75/data_to_gpu/motif4_feature.pickle','wb') as f:
    pickle.dump(motif4_feature,f)

save...
save...
save...
save...
save...


In [0]:
def build_motif_feature(motif_feature):
    res = []
    flag = 0
    for val in motif_feature:
        #val:[[],[],[],[]]*5
        flag += 1
        pre = val[0][1]
        if val[0][0] != []:
            pre += val[0][2] + val[0][3]
        else:
            pre += [0]*6
        line = np.array(pre)
        res += [line]
        if flag == 1:print 'motif_feature: ',line.shape
        if flag % 100 == 0:print '%d / %d\r' %(flag, len(motif_feature)),
    r = np.array(res)
    return r

In [0]:
m1 = build_motif_feature(motif1_feature)
m2 = build_motif_feature(motif2_feature)
m3 = build_motif_feature(motif3_feature)
m4 = build_motif_feature(motif4_feature)

motif_feature:  (106,)
motif_feature:  (106,)
motif_feature:  (106,)
motif_feature:  (106,)


In [0]:
data_x = np.concatenate([co_feature,bid1_embed,bid2_embed,m4,m3],axis = 1)

In [0]:
data_x = np.array(data_x)
print data_x.shape
train_y = label[train_index]
test_y = label[test_index]
train_x = data_x[train_index,:]
test_x = data_x[test_index,:]

(16393, 632)


In [0]:
import xgboost as xgb
xgbmodel = xgb.XGBClassifier(n_estimators=100, 
                             silent=True, 
                             objective='binary:logistic',
                             max_depth=2, 
                             learning_rate=0.1)
xgbmodel.fit(train_x,train_y,early_stopping_rounds=10,eval_set=[(test_x,test_y)],eval_metric="error")

[0]	validation_0-error:0.311585
Will train until validation_0-error hasn't improved in 10 rounds.
[1]	validation_0-error:0.311585
[2]	validation_0-error:0.311585
[3]	validation_0-error:0.311585
[4]	validation_0-error:0.311585
[5]	validation_0-error:0.30061
[6]	validation_0-error:0.279878
[7]	validation_0-error:0.279878
[8]	validation_0-error:0.281098
[9]	validation_0-error:0.277439
[10]	validation_0-error:0.277439
[11]	validation_0-error:0.281098
[12]	validation_0-error:0.280488
[13]	validation_0-error:0.277439
[14]	validation_0-error:0.278049
[15]	validation_0-error:0.279268
[16]	validation_0-error:0.27561
[17]	validation_0-error:0.27561
[18]	validation_0-error:0.27439
[19]	validation_0-error:0.272561
[20]	validation_0-error:0.27561
[21]	validation_0-error:0.277439
[22]	validation_0-error:0.272561
[23]	validation_0-error:0.273171
[24]	validation_0-error:0.275
[25]	validation_0-error:0.272561
[26]	validation_0-error:0.272561
[27]	validation_0-error:0.272561
[28]	validation_0-error:0.27

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=2, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [0]:
score = xgbmodel.predict_proba(test_x)  #预测分类概率
predict_y = xgbmodel.predict(test_x)
pp = [int(x[1]+0.5) for x in score]
get_score(test_y,score[:,1],pp)

acc:  0.7451219512195122 auc:  0.8173477219514527 f1_score:  0.7508939213349225 
precision:  0.7816377171215881 recall:  0.7224770642201835


/home/bil/anaconda2/lib/python2.7/site-packages/sklearn/preprocessing/label.py:171: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


# path/pra

In [0]:
import time
import pickle
st = time.time()
with open('/home/bil/huangtao/project/data/competition/data_75/share_visit.pickle') as f:
    share_visit_res=pickle.load(f)
print time.time() - st
print len(share_visit_res)
import time
st = time.time()
with open('/home/bil/huangtao/project/data/competition/data_75/negtive.pickle') as f:
    negtive=pickle.load(f)
print time.time() - st
print len(negtive)

0.0688719749451
8970
0.0371761322021
7423


In [0]:
tag_num_dict = {}
flag = 0
print 'sample...'
path_dict = {}
for pair in share_visit_res.keys() + negtive:
    flag+=1
    bid1 = pair.split('#')[0]
    bid2 = pair.split('#')[1]
    try:
        motifs = sfe.sample_3len_path(bid1, bid2, 500)
        path = motifs[:5]
        path_dict[pair] = path
    except:
        path_dict[pair] = [[]]*5
        continue
    for motif in motifs:
        if motif == []:continue
        tag1 = get_tag1(motif[0])
        tag2 = get_tag1(motif[-1])
        key = tag1 + '#' + tag2
        if key in tag_num_dict:
            tag_num_dict[key] += 1
        else:
            tag_num_dict[key] = 1
    if flag % 100 == 0:
        print '%d / %d\r' %(flag, len(share_visit_res.keys() + negtive)),

sample...


In [0]:
path_dict_res = {}
flag = 0
print 'make data...'
for pair in path_dict:
    flag += 1
    paths = path_dict[pair]

    for path in paths:
        if path == []:
            if pair in path_dict_res:
                path_dict_res[pair] += [[[], [], []]]
            else:
                path_dict_res[pair] = [[path, [], []]]
        else:
            co_sq = [bid_co(path[0], path[1]), bid_co(path[1], path[2]), bid_co(path[2], path[0])]
            dis_sq = [bid_dis(path[0], path[1]), bid_dis(path[1], path[2]), bid_dis(path[2], path[0])]
            if pair in path_dict_res:
                path_dict_res[pair] += [[path, co_sq, dis_sq]]
            else:
                path_dict_res[pair] = [[path, co_sq, dis_sq]]
    if flag % 100 == 0:
        print '%d / %d\r' %(flag, len(path_dict)),

make data...


In [0]:
for i in path_dict_res:
    co =path_dict_res[i][0][1]
    if len(co) != 3 and co != []:
        print co
        break

In [0]:
i = 0
for j in path_dict_res:
    if path_dict_res[j][0][0] != []:i+=1
print i

15449


In [0]:
path_dict = path_dict_res

In [0]:
motif1_sort = sorted(tag_num_dict.iteritems(),key = lambda x:x[1], reverse=True)
ordinary_list = []
for i in motif1_sort:
    ordinary_list += [i[0]]
    if len(ordinary_list) == 100:break
#for i in ordinary_list:print i

In [0]:
flag = 0
motif_dict = {}
ordinary_dict = {}
print 'get_ordinary...'
for pair in share_visit_res:
    flag+=1
    bid1 = pair.split('#')[0]
    bid2 = pair.split('#')[1]
    temp_dict = {}
    for i in ordinary_list:
        temp_dict[i] = 0
    try:
        motifs, temp_dict = sfe.get_ordinary_3len_path(bid1, bid2, 5, temp_dict, 1000)#n个motif，搜索m次
    except:
        motifs = [[]] * 5
    motif_dict[pair] = motifs
    ordinary_dict[pair] = temp_dict
    if flag % 100 == 0:
        print '%d / %d\r' %(flag, len(share_visit_res)),

get_ordinary...


In [0]:
motif_dict_res = {}
flag = 0
print 'make data..'
for pair in motif_dict:
    flag += 1
    temp_dict = ordinary_dict[pair]
    pre = []
    for p in ordinary_list:
        pre += [temp_dict[p]]
    motif_dict_res[pair] = pre
    if flag % 100 == 0:
        print '%d / %d\r' %(flag, len(motif_dict)),

make data..


In [0]:
flag = 0
motif_neg_dict = {}
ordinary_neg_dict = {}
print 'get ordinary...'
for pair in negtive:
    flag+=1
    bid1 = pair.split('#')[0]
    bid2 = pair.split('#')[1]
    temp_dict = {}
    for i in ordinary_list:
        temp_dict[i] = 0
    try:
        motifs, temp_dict = sfe.get_ordinary_3len_path(bid1, bid2, 5, temp_dict, 1000)#n个motif，搜索m次
    except:
        motifs = [[]] * 5
    motif_neg_dict[pair] = motifs
    ordinary_neg_dict[pair] = temp_dict
    if flag % 100 == 0:
        print '%d / %d\r' %(flag, len(negtive)),

motif_negtive_dict = {}
flag = 0
print 'make data...'
for pair in motif_neg_dict:
    flag += 1
    temp_dict = ordinary_neg_dict[pair]
    pre = []
    for p in ordinary_list:
        pre += [temp_dict[p]]
    motif_negtive_dict[pair] = pre
    if flag % 100 == 0:
        print '%d / %d\r' %(flag, len(motif_neg_dict)),

get ordinary...
make data...


In [0]:
pra_dict = motif_dict_res
pra_negtive_dict = motif_negtive_dict

In [0]:
pra_feature = []
flag = 0
for i in range(len(data_cut)):
    flag +=1
    bid1 = data_cut.loc[i,'bid1']
    bid2 = data_cut.loc[i,'bid2']
    pair = bid1 + '#' + bid2
    if pair in pra_dict:
        val = pra_dict[pair]
    else:
        val = pra_negtive_dict[pair]
    pra_feature += [val]
    if flag % 100 == 0:print '%d / %d\r' %(flag, len(data_cut)),

In [0]:
data_x = np.concatenate([pra_feature],axis = 1)

In [0]:
data_x = np.array(data_x)
print data_x.shape
train_y = label[train_index]
test_y = label[test_index]
train_x = data_x[train_index,:]
test_x = data_x[test_index,:]

(16393, 100)


In [0]:
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
print '逻辑回归模型:\n',model

逻辑回归模型:
LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)


In [0]:
model.fit(train_x,train_y)
score = model.predict_proba(test_x)  #预测分类概率
predict_y = model.predict(test_x)
acc = model.score(test_x,test_y)
pp = [int(x[1]+0.5) for x in score]
get_score(test_y,score[:,1],pp)

acc:  0.7067073170731707 auc:  0.7716762531058868 f1_score:  0.7168922895821072 
precision:  0.7363966142684402 recall:  0.698394495412844


In [0]:
import xgboost as xgb
xgbmodel = xgb.XGBClassifier(n_estimators=100, 
                             silent=True, 
                             objective='binary:logistic',
                             max_depth=2, 
                             learning_rate=0.1)
xgbmodel.fit(train_x,train_y,early_stopping_rounds=10,eval_set=[(test_x,test_y)],eval_metric="error")

[0]	validation_0-error:0.311585
Will train until validation_0-error hasn't improved in 10 rounds.
[1]	validation_0-error:0.311585
[2]	validation_0-error:0.311585
[3]	validation_0-error:0.311585
[4]	validation_0-error:0.292073
[5]	validation_0-error:0.292073
[6]	validation_0-error:0.277439
[7]	validation_0-error:0.287195
[8]	validation_0-error:0.277439
[9]	validation_0-error:0.277439
[10]	validation_0-error:0.278049
[11]	validation_0-error:0.278659
[12]	validation_0-error:0.278049
[13]	validation_0-error:0.278659
[14]	validation_0-error:0.278049
[15]	validation_0-error:0.277439
[16]	validation_0-error:0.27622
[17]	validation_0-error:0.27622
[18]	validation_0-error:0.276829
[19]	validation_0-error:0.276829
[20]	validation_0-error:0.278049
[21]	validation_0-error:0.27622
[22]	validation_0-error:0.277439
[23]	validation_0-error:0.279878
[24]	validation_0-error:0.27622
[25]	validation_0-error:0.27622
[26]	validation_0-error:0.27622
Stopping. Best iteration:
[16]	validation_0-error:0.27622



XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=2, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [0]:
score = xgbmodel.predict_proba(test_x)  #预测分类概率
predict_y = xgbmodel.predict(test_x)
pp = [int(x[1]+0.5) for x in score]
get_score(test_y,score[:,1],pp)

acc:  0.723780487804878 auc:  0.7959200891150611 f1_score:  0.7324276432368576 
precision:  0.7551766138855055 recall:  0.7110091743119266


/home/bil/anaconda2/lib/python2.7/site-packages/sklearn/preprocessing/label.py:171: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


# path

In [0]:
#------------motif1---------------
path_node = []#[poi*3,co_sq, dis_sq]*5
path_feature = []
flag = 0
for i in range(len(data_cut)):
    flag +=1
    bid1 = data_cut.loc[i,'bid1']
    bid2 = data_cut.loc[i,'bid2']
    pair = bid1 + '#' + bid2
    val = path_dict[pair]#[poi*3,co_sq, dis_sq]*5
    paths = val
    pn = []
    pf = []
    for path in paths:
        
        if path[0] != []:
            pn += [[poi_embed_dict[path[0][0]][0], poi_embed_dict[path[0][1]][0], poi_embed_dict[path[0][2]][0]]]
            feature = path[1] + path[2]
        else:
            pn += [[[0] * 209] * 3]
            feature = [0] * 6
        pf += [feature]
    path_node += [pn]
    path_feature += [pf]
    if flag % 100 == 0:print '%d / %d\r' %(flag, len(data_cut)),

In [0]:
path_node = np.array(path_node)
path_feature = np.array(path_feature)
print path_node.shape, path_feature.shape

(16393, 5, 3, 209) (16393, 5, 6)


In [0]:
print 'save...'
with open('/home/bil/huangtao/project/data/competition/data_75/data_to_gpu/path_node.pickle','wb') as f:
    pickle.dump(path_node,f)
print 'save...'
with open('/home/bil/huangtao/project/data/competition/data_75/data_to_gpu/path_feature.pickle','wb') as f:
    pickle.dump(path_feature,f)

save...
save...


In [0]:
path_f = path_feature[:,0,:]
path_n = path_node[:,0,:,:].reshape(-1,3*209)

In [0]:
data_x = np.concatenate([co_feature, bid1_embed, bid2_embed, path_f, region1],axis = 1)

In [0]:
import xgboost as xgb
xgbmodel = xgb.XGBClassifier(n_estimators=100, 
                             silent=True, 
                             objective='binary:logistic',
                             max_depth=2, 
                             learning_rate=0.1)
xgbmodel.fit(train_x,train_y,early_stopping_rounds=10,eval_set=[(test_x,test_y)],eval_metric="error")

[0]	validation_0-error:0.311585
Will train until validation_0-error hasn't improved in 10 rounds.
[1]	validation_0-error:0.311585
[2]	validation_0-error:0.311585
[3]	validation_0-error:0.311585
[4]	validation_0-error:0.292073
[5]	validation_0-error:0.292073
[6]	validation_0-error:0.277439
[7]	validation_0-error:0.287195
[8]	validation_0-error:0.277439
[9]	validation_0-error:0.277439
[10]	validation_0-error:0.278049
[11]	validation_0-error:0.278659
[12]	validation_0-error:0.278049
[13]	validation_0-error:0.278659
[14]	validation_0-error:0.278049
[15]	validation_0-error:0.277439
[16]	validation_0-error:0.27622
[17]	validation_0-error:0.27622
[18]	validation_0-error:0.276829
[19]	validation_0-error:0.276829
[20]	validation_0-error:0.278049
[21]	validation_0-error:0.27622
[22]	validation_0-error:0.277439
[23]	validation_0-error:0.279878
[24]	validation_0-error:0.27622
[25]	validation_0-error:0.27622
[26]	validation_0-error:0.27622
Stopping. Best iteration:
[16]	validation_0-error:0.27622



XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=2, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [0]:
score = xgbmodel.predict_proba(test_x)  #预测分类概率
predict_y = xgbmodel.predict(test_x)
pp = [int(x[1]+0.5) for x in score]
get_score(test_y,score[:,1],pp)

acc:  0.723780487804878 auc:  0.7959200891150611 f1_score:  0.7324276432368576 
precision:  0.7551766138855055 recall:  0.7110091743119266


/home/bil/anaconda2/lib/python2.7/site-packages/sklearn/preprocessing/label.py:171: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
